# Mapping the phase space of a jet


## 1-D Unfolding  of $\tau_{1}^{(\beta)}$ measured with $E$-scheme recombination, with a purity&stability study for each observables' binning


#### Firstly , import the ROOT data analysis framework which will be used to read the data files and unfold the final result. 

In [1]:
import ROOT
import array as array
from array import array
import os
import glob
from ROOT import TH1D, TH2D, TTree,TFile, TObject, TCanvas, TGraph, TH1F, TH2F, TMath
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns



Welcome to JupyROOT 6.18/00


In [2]:
#!/usr/bin/env python
import argparse, os, shutil, sys
from datasets import *
from array import array
import numpy as np
from DrawHistogram import plotSimpleComparison
#sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrStyle
####gReset()
ROOT.gROOT.SetBatch()
ROOT.gROOT.ForceStyle()
tdrStyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)
ROOT.TH1.SetDefaultSumw2()


In [3]:
%jsroot on

In [4]:
def Fill2D(h0,h,binx,biny):
    Nx=h0.GetNbinsX()
    Ny=h0.GetNbinsY()
    hx=h0.ProjectionX()
    hy=h0.ProjectionY()
    content=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            content[i][j]=0
    for i in range(Nx):
        for j in range(Ny):
            centerx=hx.GetBinCenter(i+1)
            indexx=np.digitize(np.array([centerx]),binx)
            centery=hy.GetBinCenter(j+1)
            indexy=np.digitize(np.array([centery]),biny)
            if indexx[0] < len(binx) and indexy[0] < len(biny):
                content[indexx[0]-1][indexy[0]-1]+=h0.GetBinContent(i+1,j+1)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            h.SetBinContent(i+1,j+1,content[i][j])
            
def purity_stability(h0,bins):
    
    h=ROOT.TH2D("",";;",bins.shape[0]-1,(bins),bins.shape[0]-1,(bins))
    Fill2D(h0,h,bins,bins)
    hx=h.ProjectionX()
    hy=h.ProjectionY()
    N=hx.GetNbinsX()
    
    nreco_pergenbin=[]
    nreco_genanywhere=[]
    ngen_recoanywhere=[]
    for i in range(N):
        nreco_pergenbin.append(h.GetBinContent(i+1,i+1))
        nreco_genanywhere.append(hy.GetBinContent(i+1))
        ngen_recoanywhere.append(hx.GetBinContent(i+1))
    
    nreco_pergenbin=np.array(nreco_pergenbin)
    nreco_genanywhere=np.array(nreco_genanywhere)
    ngen_recoanywhere=np.array(ngen_recoanywhere)
    
    purity_arr=1.0*nreco_pergenbin/nreco_genanywhere
    stability_arr=1.0*nreco_pergenbin/ngen_recoanywhere
    
    for i in xrange(0,purity_arr.shape[0]):
        if np.isinf(purity_arr[i])==True or np.isnan(purity_arr[i])==True: purity_arr[i]=0.
        if np.isinf(stability_arr[i])==True or np.isnan(stability_arr[i])==True: stability_arr[i]=0.
            
    purity=ROOT.TH1D("Purity",";;",bins.shape[0]-1,(bins))
    stability=ROOT.TH1D("Stability",";;",bins.shape[0]-1,(bins))
    
    for i in range(N):
        purity.SetBinContent(i+1,purity_arr[i])
        stability.SetBinContent(i+1,stability_arr[i])
    
    purity.SetLineWidth(2)
    purity.SetLineStyle(2)
    stability.SetLineWidth(2)
    stability.SetLineStyle(2)
    
    return purity,stability


In [5]:
def loadHistograms( samples, variables, sel, sysUnc=[], isMC=True ):
    """docstring for loadHistograms"""

    SYSUNC = [ '_nom' ] + [ s+u for u in ['Down', 'Up'] for s in sysUnc ]
    
    allHistos = {}
    for var in variables:
        for isam in samples:
            tmpList = [ 'recoJet'+var+syst+sel for syst in SYSUNC ]
            if isMC: tmpList = tmpList + [ 'genJet'+var+sel ] + [ 'respJet'+var+syst+sel for syst in SYSUNC ]
            for ih in tmpList:
                #print var, isam, ih
                allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
                
                if isMC:
                    tmpIsam = 'TT' if isam.startswith('data') else isam
                    MCScale = checkDict( tmpIsam, dictSamples )['XS'] * lumi / checkDict( tmpIsam, dictSamples )['2016']['nGenWeights']
                    allHistos[isam+'_'+ih].Scale( MCScale )
                
                if not ih.startswith('resp'):
                    if len(variables[var])==1:
                        if ih.startswith('reco'):
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                            allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                            allHistos[isam+'_'+ih+'_genBin'].Rebin( variables[var][0] )
                            allHistos[isam+'_'+ih].Rebin( int(variables[var][0]/2.) )
                        else: allHistos[isam+'_'+ih].Rebin( variables[var][0] )
                        #print ih, variables[var][0]
                    else:
                        if ih.startswith('reco'):
                            newRecoBins = array('d',[])#np.empty(2*variables[var].shape[0]-1)#np.array(sorted([ np.round((variables[var][i]+variables[var][i+1])/2.,3) for i in range(len(variables[var])-1) ] + variables[var]))
                            for i in range(0,len(variables[var])-1):
                                #print (variables[var][i]+variables[var][i+1])/2., variables[var][i], variables[var][i+1]
                                newRecoBins.append(np.round((variables[var][i]+variables[var][i+1])/2.,3))
                            
                            #print newRecoBins, variables[var]    
                            #newRecoBins = np.concatenate((newRecoBins,variables[var]))
                            #newRecoBins = np.sort(newRecoBins)
                            newRecoBins.extend(variables[var])
                            newRecoBins = array('d', sorted(newRecoBins))
                            print ("New variable bins:", newRecoBins, ih, variables[var])
                            allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                            allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih+'_genBin'].Rebin(len(variables[var])-1, allHistos[isam+'_'+ih].GetName()+"_Rebin_genBin", array( 'd', variables[var])); 
                            #allHistos[isam+'_'+ih+'_genBin'].SetDirectory(0)
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                            allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(newRecoBins)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', newRecoBins ) )
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                        else:
                            #print ih,  len(variables[var])-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', variables[var])
                            allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(variables[var])-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', variables[var] ) )
                            #allHistos[isam+'_'+ih].SetDirectory(0)
                else:
                    if len(variables[var])==1: allHistos[isam+'_'+ih].Rebin2D( variables[var][0], int(variables[var][0]/2.) )
                    else:
                        #newRecoBins = sorted([ (variables[var][i]+variables[var][i+1])/2. for i in range(len(variables[var])-1) ] + variables[var])
                        newRecoBins = array('d',[])#np.empty(2*variables[var].shape[0]-1)#np.array(sorted([ np.round((variables[var][i]+variables[var][i+1])/2.,3) for i in range(len(variables[var])-1) ] + variables[var]))
                        for i in range(0,len(variables[var])-1):
                            #print (variables[var][i]+variables[var][i+1])/2., variables[var][i], variables[var][i+1]
                            newRecoBins.append(np.round((variables[var][i]+variables[var][i+1])/2.,3))

                        #print newRecoBins, variables[var]    
                        #newRecoBins = np.concatenate((newRecoBins,variables[var]))
                        #newRecoBins = np.sort(newRecoBins)
                        newRecoBins.extend(variables[var])
                        newRecoBins = sorted(newRecoBins)
                        print ("New variable bins TH2F:", newRecoBins)
                        #### fancy way to create variable binning TH2D
                        tmpHisto = TH2F( allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", len(variables[var])-1, array( 'd', variables[var]), len(newRecoBins)-1, array( 'd', newRecoBins) )

                        tmpArrayContent = np.zeros((len(variables[var]), len(newRecoBins)))
                        tmpArrayError = np.zeros((len(variables[var]), len(newRecoBins)))

                        xaxis = allHistos[isam+'_'+ih].GetXaxis() 
                        yaxis = allHistos[isam+'_'+ih].GetYaxis()
                        
                        for j in range(1, yaxis.GetNbins()+1):
                            for i in range(1, xaxis.GetNbins()+1): 
                                tmpHisto.Fill(xaxis.GetBinCenter(i),yaxis.GetBinCenter(j),allHistos[isam+'_'+ih].GetBinContent(i,j)) 

                        allHistos[isam+'_'+ih] = tmpHisto
                        #allHistos[isam+'_'+ih].SetDirectory(0)
                
                    ##### For tests, projections directly from 2D
                    #allHistos[isam+'_genJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionY()
                    #allHistos[isam+'_genJetfrom_'+ih].SetDirectory(0)
                    #allHistos[isam+'_recoJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionX()
                    #allHistos[isam+'_recoJetfrom_'+ih].SetDirectory(0)
                    
                allHistos[isam+'_'+ih+'_Normalized'] = allHistos[isam+'_'+ih].Clone()
                #allHistos[isam+'_'+ih+'_Normalized'].SetDirectory(0)
                try: allHistos[isam+'_'+ih+'_Normalized'].Scale( 1/allHistos[isam+'_'+ih+'_Normalized'].Integral() )
                except ZeroDivisionError: continue
                    
                histos=allHistos.copy()
            #samples[isam][0].Close()
    return histos


In [6]:
ROOT.TH1.SetDefaultSumw2()


#### Below the MC and Data events are read from ROOT trees which only contain events meeting the selection criteria set forth in this __[script](https://github.com/kaustuvdatta/jetObservables/blob/102X/python/nSubProducer_gen_reco.py)__ and thereafter in the nSubExtractor class

In [7]:
lumi=35920

In [8]:
variables = {}
variables[ '_tau_0p5_0' ] = array('d', [0., 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1. ])
#variables[ '_tau_1_0' ] = array('d',[0., 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.52, 0.58, 0.64, 0.8])#[0. 0.25 0.30625 0.3625 0.418750.475 0.53125 0.5875 0.64375 0.7 0.8 ][0., 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.65]#[ 6]
#variables[ '_tau_2_0' ] = [ 3]
#variables[ '_tau_0p5_1' ] = [10] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_1' ] = [ 8]
#variables[ '_tau_2_1' ] = [ 5]
#variables[ '_tau_0p5_2' ] = [8] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_2' ] = [ 5]
#variables[ '_tau_2_2' ] = [ 4]

#variables[ '_tau_0p5_3' ] = [8] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_3' ] = [ 5]
#variables[ '_tau_2_3' ] = [0., 0.006, 0.012, 0.018, 0.024, 0.03, 0.04, 0.055, 0.075, 0.11] #[4]#[0., 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02, 0.0225, 0.025]

sysUncert = [ '_jesTotal', '_jer', '_pu' ]
print (variables)

{'_tau_0p5_0': array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])}


In [9]:
sel= '_topSel'
process = 'data'
version = 'v01_fin'

In [10]:
dataFile = {}
dataFile['data'] = [ TFile('Histos_SysUnc/jetObservables_histograms_SingleMuonRun2016ALL.root' ), 'Data', 'kBlack' ]

In [11]:
sigFiles = {}
altsigFiles = {}
sigFiles['TT'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TT_TuneCUETP8M2T4_13TeV-powheg-pythia8.root'), 'ttbar (powheg)', 'kRed' ]
altsigFiles['TTJets'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8.root'), 'ttbar (MG)', 'kBlue' ]

In [12]:
bkgFiles = {}
bkgFiles['ST_s-channel_4f_InclusiveDecays'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_s-channel_4f_InclusiveDecays_13TeV-amcatnlo-pythia8.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_antitop'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_t-channel_antitop_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_top'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_t-channel_top_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_antitop'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M2T4.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_top'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M2T4.root' ), 'Single top', 'kMagenta' ]
#bkgFiles['WJets'] = [ TFile( inputFolder+'/jetObservables_histograms_WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8.root' ), 'WJets', 'kCyan' ]
###bkgFiles[] = [ '', TFile( inputFolder+'/jetObservables_histograms_'+ibkg+'.root' ), '', 'kMagenta' ]

In [13]:
dataHistos1 = loadHistograms( dataFile, variables, sel='_topSel', isMC=False )
signalHistos1 = loadHistograms( sigFiles, variables, sel='_topSel', sysUnc=sysUncert )
bkgHistos1 = loadHistograms( bkgFiles, variables, sel='_topSel', sysUnc=sysUncert ) 

New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_nom_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_nom_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_jesTotalDown_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_jerDown_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8,

New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_nom_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_jesTotalDown_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5,

In [14]:
dataHistos2 = loadHistograms( dataFile, variables, sel='_topSel', isMC=False )
signalHistos2 = loadHistograms(altsigFiles, variables, sel='_topSel', sysUnc=sysUncert)
bkgHistos2 = loadHistograms( bkgFiles, variables, sel='_topSel', sysUnc=sysUncert ) 

New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_nom_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_nom_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_jesTotalDown_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_jerDown_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8,

New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins TH2F: [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_nom_topSel array('d', [0.0, 0.3, 0.38, 0.46, 0.54, 0.62, 0.7, 0.8, 1.0])
New variable bins: array('d', [0.0, 0.15, 0.3, 0.34, 0.38, 0.42, 0.46, 0.5, 0.54, 0.58, 0.62, 0.66, 0.7, 0.75, 0.8, 0.9, 1.0]) recoJet_tau_0p5_0_jesTotalDown_topSel array('d', [0.0, 

Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_nom_topSelST_s-channel_4f_InclusiveDecays_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_nom_topSelST_s-channel_4f_InclusiveDecays_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_jesTotalDown_topSelST_s-channel_4f_InclusiveDecays_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_jesTotalDown_topSelST_s-channel_4f_InclusiveDecays_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_jerDown_topSelST_s-channel_4f_InclusiveDecays_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_jerDown_topSelST_s-channel_4f_InclusiveDecays_Rebin (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: respJet_tau_0p5_0_puDown_topSelST_s-channel_4f_InclusiveDecays_R

In [17]:
for ivar in variables:
    
    print ('|------> Unfolding '+ivar)

    ######## Cross check: plotting data vs all MC Scaled
    print ('|------> Cross check: plotting data vs all MC')
    allBkgHisto1 = dataHistos1['data_recoJet'+ivar+'_nom'+sel].Clone()
    allBkgHisto1.Reset()
    allBkgHisto1Norm = allBkgHisto1.Clone()
    allBkgHisto1GenBin = dataHistos1['data_recoJet'+ivar+'_nom'+sel+'_genBin'].Clone()
    allBkgHisto1GenBin.Reset()
    for ibkg in bkgHistos1:
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel): allBkgHisto1.Add( bkgHistos1[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_genBin'): allBkgHisto1GenBin.Add( bkgHistos1[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'): allBkgHisto1Norm.Add( bkgHistos1[ibkg].Clone() )
    allMCHisto1 = allBkgHisto1.Clone()
    allMCHisto1.Add( signalHistos1[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel ].Clone() )
    plotSimpleComparison( dataHistos1['data_recoJet'+ivar+'_nom'+sel].Clone(), 'data', allMCHisto1, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom", rebinX=variables[ivar][0], version=sel+'_'+version  )
    allMCHisto1Norm = allBkgHisto1Norm.Clone()
    allMCHisto1Norm.Add( signalHistos1[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized" ].Clone() )
    plotSimpleComparison( dataHistos1['data_recoJet'+ivar+'_nom'+sel+"_Normalized"].Clone(), 'data', allMCHisto1Norm, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom_Normalized", rebinX=variables[ivar][0], version=sel+'_'+version  )

    ######## Cross check: plotting response matrix
    #tdrStyle.SetPadRightMargin(0.12)
    print ('|------> Cross check: plotting response matrix for signal')
    can_1_2D = ROOT.TCanvas(ivar+'can_1_2D', ivar+'can_1_2D', 750,600 )
    signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].SetStats(0)
    signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].Draw("colz")
    can_1_2D.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_1_2D, 4, 0)
    can_1_2D.Draw()
    can_1_2D.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'_responseMatrix'+version+'.png')
    can_1_2DNorm = ROOT.TCanvas(ivar+'can_1_2DNorm', ivar+'can_1_2DNorm', 750, 600 )
    signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].SetStats(0)
    signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].Draw("colz")
    can_1_2DNorm.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_1_2DNorm, 4, 0)
    can_1_2DNorm.Draw()
    can_1_2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'Normalized_responseMatrix'+version+'.png')

    ####### Removing bkgs from data
    dataMinusBkgs1 = dataHistos1[ 'data_recoJet'+ivar+'_nom'+sel ].Clone()
    dataMinusBkgs1.Add( allBkgHisto1.Clone(), -1 )
    dataMinusBkgs1.Scale( 1/dataMinusBkgs1.Integral() )
    dataMinusBkgs1GenBin = dataHistos1[ 'data_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
    dataMinusBkgs1GenBin.Add( allBkgHisto1GenBin.Clone(), -1 )
    dataMinusBkgs1GenBin.Scale( 1/dataMinusBkgs1GenBin.Integral() )

    ######## TUnfold part
    print ('|------> TUnfolding starts:')

    ##### Defining options for TUnfold
    tunfolder1 = ROOT.TUnfoldDensity(
                                        signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'], ### response matrix
                                        #signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel], ### response matrix
                                        ROOT.TUnfold.kHistMapOutputHoriz,  #### kHistMapOutputVert if x->reco and y->gen, kHistMapOutputHoriz if x->gen and y->reco
                                        ROOT.TUnfold.kRegModeCurvature,   ##### Regularization Mode : ROOT.TUnfold.kRegModeCurvature regularizes based on the 2nd derivative of the output. More information wrt the other options can_1_ be gained from reading the source code
                                        ROOT.TUnfold.kEConstraintNone,    ##### Constraint : TUnfold.kEConstraintNone meaning we do not constrain further, the other option is to force constraint of area. (Need to look into this!!)
                                        ROOT.TUnfoldDensity.kDensityModeBinWidth  ##### Density Mode: ROOT.TUnfoldDensity.kDensityModeBinWidth uses the bin width to normalize the event rate in a given bin, accounting for non-uniformity in bin widths as discussed in section 7.2.1 of the TUnfold paper
                                        )

    ##### Defining input (data recoJet )
    print ('|------> TUnfolding adding input:')
    #tunfolder1.SetInput( dataHistos1[ 'data_recoJet'+ivar+'_nom'+sel ].Clone() )
    tunfolder1.SetInput( dataMinusBkgs1 )

    ###### Removing bkgs from data using TUnfold. Better to subtract bkgs beforehand
    #for ibkg in bkgHistos1:
    #    if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'):
    #        print '|--------> Removing this bkg: ', ibkg
    #        tunfolder1.SubtractBackground( bkgHistos1[ibkg], ibkg )

    ###### Adding SYS unc
    if len(sysUncert)>0 :
        print ('|------> TUnfolding adding uncert:')
        for sys in sysUncert:
            #plotSysComparison( signalHistos1[next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized"],
            #                    signalHistos1[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Up'+sel+"_Normalized"],
            #                    signalHistos1[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Down'+sel+"_Normalized"],
            #                    ivar+'_'+next(iter(sigFiles)),
            #                    sys.split('_')[1],
            #                    version=sel+'_'+version
            #                    )
            for upDown in [ 'Up', 'Down' ]:
                print (sys+upDown)
                tunfolder1.AddSysError(
                                    signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"],
                                    sys+upDown,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                    )
                can_1_2DNorm = ROOT.TCanvas(ivar+'can_1_2DNorm'+sys+upDown, ivar+'can_1_2DNorm'+sys+upDown, 750, 600 )
                signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].SetStats(0)
                signalHistos1[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].Draw("colz")
                can_1_2DNorm.SetRightMargin(0.13)
                CMS_lumi.extraText = "  Preliminary"
                CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
                #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
                CMS_lumi.relPosX = 0.11
                CMS_lumi.CMS_lumi(can_1_2DNorm, 4, 0)
                can_1_2DNorm.Draw()
                can_1_2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+sys+upDown+'Normalized_responseMatrix'+version+'.png')

    ###### Running the unfolding
    print ('|------> TUnfolding doUnfold:')
    tunfolder1.DoUnfold(0)

    ###### Regularization
#        nScan_1_=50
#        tauMin=0.0
#        tauMax=0.0
#        iBest=0
#
#        logTauX = ROOT.MakeNullPointer(ROOT.TSpline)
#        logTauY = ROOT.MakeNullPointer(ROOT.TSpline)
#        lCurve = ROOT.MakeNullPointer(ROOT.ROOT.TGraph)
#        ## this method scan_1_s the parameter tau and finds the kink in the L curve finally, the unfolding is done for the best choice of tau
#        tunfolder1.Scan_1_Lcurve(nScan_1_,tauMin,tauMax,lCurve,logTauX,logTauY)
    #########################
    ROOT.gStyle.SetOptStat(0)

    ##### Get output of unfolding
    unfoldHisto1 = tunfolder1.GetOutput("unfoldHisto")

    #### Get various covariances
    print ('|------> TUnfolding covariances')
    cov1 = tunfolder1.GetEmatrixTotal("cov", "Covariance Matrix")
    cov1_uncorr = tunfolder1.GetEmatrixSysUncorr("cov1_uncorr", "Covariance Matrix from Uncorrelated Uncertainties")
    cov1_uncorr_data = tunfolder1.GetEmatrixInput("cov1_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data")
    unfoldHisto1woUnc = unfoldHisto1.Clone()        # Unfolding and stat unc
    unfoldHisto1StatUnc = unfoldHisto1.Clone("unfoldHisto1StatUnc")        # Unfolding and stat unc
    unfoldHisto1TotUnc = unfoldHisto1.Clone("unfoldHisto1TotUnc")          # Total uncertainty
    for ibin in range( 0, unfoldHisto1.GetNbinsX()+1 ):
        unc1_tot = ROOT.TMath.Sqrt( cov1.GetBinContent(ibin,ibin) )
        unfoldHisto1TotUnc.SetBinContent(ibin, unc1_tot )
        unfoldHisto1.SetBinError(ibin, unc1_tot )


    ##### Get systematic shifts of output
    uncerUnfoldHisto1 = {}
    if len(sysUncert)>0 :
        print ('|------> TUnfolding uncertainties:')
        unfoldHisto1SysUnc = unfoldHisto1.Clone("unfoldHisto1SysUnc")          # Syst uncertainty
        unfoldHisto1SysUnc.Reset()
        unfoldHisto1SysUnc.SetLineStyle(2)

        for sys in sysUncert:
            for upDown in [ 'Up', 'Down' ]:
                print( sys+upDown)
                uncerUnfoldHisto1[ivar+sys+upDown] = tunfolder1.GetDeltaSysSource(sys+upDown, "unfoldHisto_"+sys+upDown+"shift", "-1#sigma")
                try: uncerUnfoldHisto1[ivar+sys+upDown].SetLineStyle(2)
                except ReferenceError: uncerUnfoldHisto1.pop( ivar+sys+upDown, None )

            # Create total uncertainty and sys uncertainty plots.
            uncerUnfoldHisto1[ivar+sys+'Total'] = unfoldHisto1.Clone("unfoldHisto1SysUnc")          # Syst uncertainty
            uncerUnfoldHisto1[ivar+sys+'Total'].Reset()
            uncerUnfoldHisto1[ivar+sys+'Total'].SetLineStyle(3)
            for i in range( 0, unfoldHisto1.GetNbinsX() + 1):
                try: yup = abs( uncerUnfoldHisto1[ivar+sys+'Up'].GetBinContent(i))
                except KeyError: yup = 0
                try: ydn = abs( uncerUnfoldHisto1[ivar+sys+'Down'].GetBinContent(i))
                except KeyError: ydn = 0
                dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
                uncerUnfoldHisto1[ivar+sys+'Total'].SetBinContent(i, dy )
            unfoldHisto1SysUnc.Add( uncerUnfoldHisto1[ivar+sys+'Total'] )

    ###### Plot unfolding results
    #tdrStyle.SetPadRightMargin(0.05)
    #tdrStyle.SetPadLeftMargin(0.15)
    can_1_ = ROOT.TCanvas('can_1_', 'can_1_',  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()
    #ROOT.gStyle.SetOptStat(0)

    pad1.cd()
    
    #tmpPad1 = pad1.DrawFrame( 0, 0.3, 1, 1.7 )
    
    #tmpPad1.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(0.5)})}' )
    #tmpPad1.GetYaxis().SetTitleOffset(0.9)
    #tmpPad1.GetYaxis().SetTitleSize(28)
    #tmpPad1.GetYaxis().SetLabelSize(20)
    #tmpPad1.SetStats(0)
    
    leg=ROOT.TLegend(0.25,0.6,0.55,0.88)
    leg.SetLineColor(0)
    leg.SetLineWidth(0)
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    #leg.SetTextSize(0.8)

    #genJetHisto1 = signalHistos1[ next(iter(sigFiles))+'_genJet'+ivar+sel ].Clone()
    genJetHisto1 = signalHistos1[ next(iter(sigFiles))+'_genJet'+ivar+sel+'_Normalized' ].Clone()
    genJetHisto1.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(0.5)})}' )
    genJetHisto1.GetYaxis().SetTitleSize( 0.05 )
    genJetHisto1.GetYaxis().SetTitleOffset( 0.9 )
    #genJetHisto1.GetYaxis().CenterTitle()
    genJetHisto1.SetStats(0)
    genJetHisto1.Scale(1, 'width')  ### divide by bin width
    genJetHisto1.SetLineWidth(2)
    genJetHisto1.GetYaxis().SetLabelSize(0.037)
    genJetHisto1.SetLineColor(1)
    pad1.Modified()
    pad1.Update()

    leg.AddEntry( genJetHisto1, 'Gen-level MC', 'l' )

    #unfoldHisto1 = tunfolder1.GetOutput("unfolded")
    #if len(variables[ivar])>1: unfoldHisto1 = unfoldHisto1.Rebin( len(variables[ivar])-1, unfoldHisto1.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    #else: unfoldHisto1.Rebin( variables[ivar][0] )
    

    unfoldHisto1woUnc.Scale(1, 'width')  ### divide by bin width
    unfoldHisto1woUnc.SetMarkerStyle(0)
    unfoldHisto1woUnc.SetMarkerColor(ROOT.kRed-5)
    unfoldHisto1woUnc.SetLineColor(ROOT.kRed-5)
    unfoldHisto1woUnc.SetLineWidth(2)
    unfoldHisto1woUnc.SetStats(0)
    leg.AddEntry( unfoldHisto1woUnc, 'Unfolded, stat+unf unc', 'l' )
    
    unfoldHisto1.Scale(1, 'width')  ### divide by bin width
    unfoldHisto1.SetMarkerStyle(4)
    unfoldHisto1.SetMarkerSize(2)
    unfoldHisto1.SetMarkerColor(ROOT.kRed+1)
    unfoldHisto1.SetLineColor(ROOT.kRed+1)
    unfoldHisto1.SetStats(0)
    leg.AddEntry( unfoldHisto1, 'Unfolded, total unc', 'pl' )

    foldHisto1 = tunfolder1.GetFoldedOutput("folded") #, 'folded', 'folded', , False)
    if len(variables[ivar])>1: foldHisto1 = foldHisto1.Rebin( len(variables[ivar])-1, foldHisto1.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    else: foldHisto1.Rebin( 2 )
    foldHisto1.Scale(1, 'width')  ### divide by bin width
    foldHisto1.SetLineWidth(2)
    foldHisto1.SetLineStyle(2)
    foldHisto1.SetLineColor(8)
    foldHisto1.SetStats(0)
    leg.AddEntry( foldHisto1, 'Folded', 'l' )

    if process.startswith('data'):
        #recoJetHisto1 = signalHistos1[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_Normalized' ].Clone()
        recoJetHisto1 = signalHistos1[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
        recoJetHisto1.Scale(1/recoJetHisto1.Integral(), 'width')  ### divide by bin width
        recoJetHisto1.SetLineWidth(2)
        recoJetHisto1.SetLineStyle(2)
        recoJetHisto1.SetLineColor(ROOT.kBlue)
        leg.AddEntry( recoJetHisto1, 'Reco-level MC', 'l' )

    dataJetHisto1 = dataMinusBkgs1GenBin.Clone()
    dataJetHisto1.Scale(1/dataJetHisto1.Integral(), 'width')  ### divide by bin width
    dataJetHisto1.SetLineWidth(2)
    dataJetHisto1.SetLineStyle(2)
    dataJetHisto1.SetLineColor(ROOT.kMagenta)
    dataJetHisto1.SetStats(0)
    leg.AddEntry( dataJetHisto1, ('Data' if process.startswith('data') else 'MC Closure' ), 'l' )

    
    
    genJetHisto1.SetMaximum( 1.3*max([ genJetHisto1.GetMaximum(), unfoldHisto1.GetMaximum(), unfoldHisto1woUnc.GetMaximum(), dataJetHisto1.GetMaximum() ] )  )
    pad1.Update()
    pad1.Modified()
    
    genJetHisto1.Draw( "histe")
    unfoldHisto1.Draw( "same")
    unfoldHisto1woUnc.Draw( "e1 same")
    foldHisto1.Draw( "hist same")
    dataJetHisto1.Draw( "hist same")
    if process.startswith('data'): recoJetHisto1.Draw( "hist same")
    #ROOT.gStyle.SetOptStat(0)
    leg.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    #ROOT.gPad.SetLogy()
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.relPosX = 0.13
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    ROOT.gStyle.SetOptStat(0000000)
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    #tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1. )
    
    h1RatioUp = genJetHisto1.Clone()
    h1RatioUp.GetYaxis().SetTitle( "#frac{MC-Truth}{Unfolded}" )
    h1RatioUp.GetYaxis().SetTitleOffset( 0.6)
    h1RatioUp.GetYaxis().SetTitleSize(0.1)
    h1RatioUp.GetYaxis().SetLabelSize(0.085)
    h1RatioUp.GetXaxis().SetTitle("#tau_{1}^{(0.5)}")
    h1RatioUp.GetXaxis().SetTitleOffset( 0.8)
    h1RatioUp.GetXaxis().SetTitleSize( 0.15)
    h1RatioUp.GetXaxis().SetLabelSize(0.085)
    h1RatioUp.GetYaxis().CenterTitle()
    h1RatioUp.Divide( unfoldHisto1 )
    h1RatioUp.GetYaxis().SetRangeUser(0.,2.)
    h1RatioUp.Draw('P0')
    
    #tmpPad2.SetLabelSize(16, 'x')
    #tmpPad2.SetTitleSize(19, 'x')
    #tmpPad2.SetLabelSize(16., 'y')
    #tmpPad2.SetTitleSize(19., 'y')
    #tmpPad2.SetNdivisions(505, 'x')
    #tmpPad2.SetNdivisions(505, 'y')
    #tmpPad2.SetMarkerStyle(8)
    #h1RatioUp.SetTitle("#tau_{1}^{(0.5)}")
    #h1RatioUp.SetLabelSize(18)
    
    #h1RatioUp.GetXaxis().SetTitle( genJetHisto1.GetXaxis().GetTitle() )
    pad2.Modified()

    #pad2.Update()
    #pad2.Modified()
    #h1RatioDown.Draw('P same')
    #pad1.cd()
    #leg.Draw()
    #can_1_.Update()
    #can_1_.Modified()
    can_1_.Draw()
    can_1_.SaveAs('Plots/'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_'+version+'.png')

    can_1_Unc = ROOT.TCanvas('can_1_Unc', 'can_1_Unc',  10, 10, 750, 500 )
    can_1_Unc.SetTitle("")
    #can_1_Unc.SetLogy()
    #ROOT.gStyle.SetOptStat(0)
    legend=ROOT.TLegend(0.25,0.65,0.35,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)
    legend.SetLineStyle(0)
    legend.AddEntry( unfoldHisto1TotUnc, 'Total Unc', 'l' )

    unfoldHisto1TotUnc.SetStats(0)
    unfoldHisto1TotUnc.SetLineWidth(2)
    unfoldHisto1TotUnc.Scale( 1/(unfoldHisto1TotUnc.Integral()) )
    unfoldHisto1TotUnc.SetMaximum(1.)
    unfoldHisto1TotUnc.SetMinimum(0.000001)
    unfoldHisto1TotUnc.GetYaxis().SetTitle('Fractional Uncertainty')
    unfoldHisto1TotUnc.Draw('hist')

    if len(sysUncert)>0 :
        legend.AddEntry( unfoldHisto1SysUnc, 'Total Syst Unc', 'l' )
        unfoldHisto1SysUnc.Scale( 1/unfoldHisto1TotUnc.Integral() )
        unfoldHisto1SysUnc.SetLineWidth(2)
        unfoldHisto1SysUnc.SetStats(0)
        unfoldHisto1SysUnc.Draw("hist same")
        dummy=2
        for k in uncerUnfoldHisto1:
            if k.endswith('Total'):
                for i in sysUncert:
                    if i in k:
                        legend.AddEntry( uncerUnfoldHisto1[k], i.split('_')[1]+" Total", 'l' )
                uncerUnfoldHisto1[k].SetLineColor(dummy)
                uncerUnfoldHisto1[k].SetLineWidth(2)
                uncerUnfoldHisto1[k].SetStats(0)
                uncerUnfoldHisto1[k].Scale( 1/unfoldHisto1TotUnc.Integral() )
                uncerUnfoldHisto1[k].Draw("hist same")
                dummy=dummy+1

    legend.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    ROOT.gPad.SetLogy()
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_1_Unc, 4, 0)
    can_1_Unc.Draw()
    can_1_Unc.SaveAs('Plots/'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_UNC_'+version+'.png')


|------> Unfolding _tau_0p5_0
|------> Cross check: plotting data vs all MC
Processing....... _tau_0p5_0_fromData_TT_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png
Processing....... _tau_0p5_0_fromData_TT_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png
|------> Cross check: plotting response matrix for signal
35.92 fb^{-1}, 2016 (13 TeV)
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding starts:
|------> TUnfolding adding input:
|------> TUnfolding adding uncert:
_jesTotalUp
35.92 fb^{-1}, 2016 (13 TeV)
_jesTotalDown
35.92 fb^{-1}, 2016 (13 TeV)
_jerUp
35.92 fb^{-1}, 2016 (13 TeV)
_jerDown
35.92 fb^{-1}, 2016 (13 TeV)
_puUp
35.92 fb^{-1}, 2016 (13 TeV)
_puDown
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
35.92 fb^{-1},  2016 (13 TeV)
35.92 fb^{-1},  2016 (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_fromData_TT_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_fromData_TT_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: _tau_0p5_0can_1_2D
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_fromData_TT_topSel_responseMatrixv01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_fromData_TT_topSelNormalized_responseMatrixv01_fin.png has been created
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 3 output bins do not depend on the input data  0 8-9
Info in <TUnfold::TUnfold>: 16 input bins and 7 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (

In [19]:
for ivar in variables:
    
    print ('|------> Unfolding '+ivar)

    ######## Cross check: plotting data vs all MC Scaled
    print ('|------> Cross check: plotting data vs all MC')
    allBkgHisto2 = dataHistos2['data_recoJet'+ivar+'_nom'+sel].Clone()
    allBkgHisto2.Reset()
    allBkgHisto2Norm = allBkgHisto2.Clone()
    allBkgHisto2GenBin = dataHistos2['data_recoJet'+ivar+'_nom'+sel+'_genBin'].Clone()
    allBkgHisto2GenBin.Reset()
    for ibkg in bkgHistos2:
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel): allBkgHisto2.Add( bkgHistos2[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_genBin'): allBkgHisto2GenBin.Add( bkgHistos2[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'): allBkgHisto2Norm.Add( bkgHistos2[ibkg].Clone() )
    allMCHisto2 = allBkgHisto2.Clone()
    allMCHisto2.Add( signalHistos2[ next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel ].Clone() )
    plotSimpleComparison( dataHistos2['data_recoJet'+ivar+'_nom'+sel].Clone(), 'data', allMCHisto2, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(altsigFiles))+"_nom", rebinX=variables[ivar][0], version=sel+'_'+version  )
    allMCHisto2Norm = allBkgHisto2Norm.Clone()
    allMCHisto2Norm.Add( signalHistos2[ next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized" ].Clone() )
    plotSimpleComparison( dataHistos2['data_recoJet'+ivar+'_nom'+sel+"_Normalized"].Clone(), 'data', allMCHisto2Norm, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(altsigFiles))+"_nom_Normalized", rebinX=variables[ivar][0], version=sel+'_'+version  )

    ######## Cross check: plotting response matrix
    #tdrStyle.SetPadRightMargin(0.12)
    print ('|------> Cross check: plotting response matrix for signal')
    can_2_2D = ROOT.TCanvas(ivar+'can_2_2D', ivar+'can_2_2D', 750,600 )
    signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+'_nom'+sel].SetStats(0)
    signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+'_nom'+sel].Draw("colz")
    can_2_2D.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_2_2D, 4, 0)
    can_2_2D.Draw()
    can_2_2D.SaveAs('Plots/'+'MG_'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(altsigFiles))+sel+'_responseMatrix'+version+'.png')
    can_2_2DNorm = ROOT.TCanvas(ivar+'can_2_2DNorm', ivar+'can_2_2DNorm', 750, 600 )
    signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].SetStats(0)
    signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].Draw("colz")
    can_2_2DNorm.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_2_2DNorm, 4, 0)
    can_2_2DNorm.Draw()
    can_2_2DNorm.SaveAs('Plots/'+'MG_'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(altsigFiles))+sel+'Normalized_responseMatrix'+version+'.png')

    ####### Removing bkgs from data
    dataMinusBkgs2 = dataHistos2[ 'data_recoJet'+ivar+'_nom'+sel ].Clone()
    dataMinusBkgs2.Add( allBkgHisto2.Clone(), -1 )
    dataMinusBkgs2.Scale( 1/dataMinusBkgs2.Integral() )
    dataMinusBkgs2GenBin = dataHistos2[ 'data_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
    dataMinusBkgs2GenBin.Add( allBkgHisto2GenBin.Clone(), -1 )
    dataMinusBkgs2GenBin.Scale( 1/dataMinusBkgs2GenBin.Integral() )

    ######## TUnfold part
    print ('|------> TUnfolding starts:')

    ##### Defining options for TUnfold
    tunfolder2 = ROOT.TUnfoldDensity(
                                        signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'], ### response matrix
                                        #signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+'_nom'+sel], ### response matrix
                                        ROOT.TUnfold.kHistMapOutputHoriz,  #### kHistMapOutputVert if x->reco and y->gen, kHistMapOutputHoriz if x->gen and y->reco
                                        ROOT.TUnfold.kRegModeCurvature,   ##### Regularization Mode : ROOT.TUnfold.kRegModeCurvature regularizes based on the 2nd derivative of the output. More information wrt the other options can_2_ be gained from reading the source code
                                        ROOT.TUnfold.kEConstraintNone,    ##### Constraint : TUnfold.kEConstraintNone meaning we do not constrain further, the other option is to force constraint of area. (Need to look into this!!)
                                        ROOT.TUnfoldDensity.kDensityModeBinWidth  ##### Density Mode: ROOT.TUnfoldDensity.kDensityModeBinWidth uses the bin width to normalize the event rate in a given bin, accounting for non-uniformity in bin widths as discussed in section 7.2.1 of the TUnfold paper
                                        )

    ##### Defining input (data recoJet )
    print ('|------> TUnfolding adding input:')
    #tunfolder2.SetInput( dataHistos2[ 'data_recoJet'+ivar+'_nom'+sel ].Clone() )
    tunfolder2.SetInput( dataMinusBkgs2 )

    ###### Removing bkgs from data using TUnfold. Better to subtract bkgs beforehand
    #for ibkg in bkgHistos2:
    #    if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'):
    #        print '|--------> Removing this bkg: ', ibkg
    #        tunfolder2.SubtractBackground( bkgHistos2[ibkg], ibkg )

    ###### Adding SYS unc
    if len(sysUncert)>0 :
        print ('|------> TUnfolding adding uncert:')
        for sys in sysUncert:
            #plotSysComparison( signalHistos2[next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized"],
            #                    signalHistos2[next(iter(altsigFiles))+'_recoJet'+ivar+sys+'Up'+sel+"_Normalized"],
            #                    signalHistos2[next(iter(altsigFiles))+'_recoJet'+ivar+sys+'Down'+sel+"_Normalized"],
            #                    ivar+'_'+next(iter(altsigFiles)),
            #                    sys.split('_')[1],
            #                    version=sel+'_'+version
            #                    )
            for upDown in [ 'Up', 'Down' ]:
                print (sys+upDown)
                tunfolder2.AddSysError(
                                    signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"],
                                    sys+upDown,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                    )
                can_2_2DNorm = ROOT.TCanvas(ivar+'can_2_2DNorm'+sys+upDown, ivar+'can_2_2DNorm'+sys+upDown, 750, 600 )
                signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].SetStats(0)
                signalHistos2[next(iter(altsigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].Draw("colz")
                can_2_2DNorm.SetRightMargin(0.13)
                CMS_lumi.extraText = "  Preliminary"
                CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
                #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
                CMS_lumi.relPosX = 0.11
                CMS_lumi.CMS_lumi(can_2_2DNorm, 4, 0)
                can_2_2DNorm.Draw()
                can_2_2DNorm.SaveAs('Plots/'+'MG_'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(altsigFiles))+sel+sys+upDown+'Normalized_responseMatrix'+version+'.png')

    ###### Running the unfolding
    print ('|------> TUnfolding doUnfold:')
    tunfolder2.DoUnfold(0)

    ###### Regularization
#        nScan_2_=50
#        tauMin=0.0
#        tauMax=0.0
#        iBest=0
#
#        logTauX = ROOT.MakeNullPointer(ROOT.TSpline)
#        logTauY = ROOT.MakeNullPointer(ROOT.TSpline)
#        lCurve = ROOT.MakeNullPointer(ROOT.ROOT.TGraph)
#        ## this method scan_2_s the parameter tau and finds the kink in the L curve finally, the unfolding is done for the best choice of tau
#        tunfolder2.Scan_2_Lcurve(nScan_2_,tauMin,tauMax,lCurve,logTauX,logTauY)
    #########################
    ROOT.gStyle.SetOptStat(0)

    ##### Get output of unfolding
    unfoldHisto2 = tunfolder2.GetOutput("unfoldHisto")

    #### Get various covariances
    print ('|------> TUnfolding covariances')
    cov2 = tunfolder2.GetEmatrixTotal("cov", "Covariance Matrix")
    cov2_uncorr = tunfolder2.GetEmatrixSysUncorr("cov2_uncorr", "Covariance Matrix from Uncorrelated Uncertainties")
    cov2_uncorr_data = tunfolder2.GetEmatrixInput("cov2_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data")
    unfoldHisto2woUnc = unfoldHisto2.Clone()        # Unfolding and stat unc
    unfoldHisto2StatUnc = unfoldHisto2.Clone("unfoldHisto2StatUnc")        # Unfolding and stat unc
    unfoldHisto2TotUnc = unfoldHisto2.Clone("unfoldHisto2TotUnc")          # Total uncertainty
    for ibin in range( 0, unfoldHisto2.GetNbinsX()+1 ):
        unc2_tot = ROOT.TMath.Sqrt( cov2.GetBinContent(ibin,ibin) )
        unfoldHisto2TotUnc.SetBinContent(ibin, unc2_tot )
        unfoldHisto2.SetBinError(ibin, unc2_tot )


    ##### Get systematic shifts of output
    uncerUnfoldHisto2 = {}
    if len(sysUncert)>0 :
        print ('|------> TUnfolding uncertainties:')
        unfoldHisto2SysUnc = unfoldHisto2.Clone("unfoldHisto2SysUnc")          # Syst uncertainty
        unfoldHisto2SysUnc.Reset()
        unfoldHisto2SysUnc.SetLineStyle(2)

        for sys in sysUncert:
            for upDown in [ 'Up', 'Down' ]:
                print( sys+upDown)
                uncerUnfoldHisto2[ivar+sys+upDown] = tunfolder2.GetDeltaSysSource(sys+upDown, "unfoldHisto_"+sys+upDown+"shift", "-1#sigma")
                try: uncerUnfoldHisto2[ivar+sys+upDown].SetLineStyle(2)
                except ReferenceError: uncerUnfoldHisto2.pop( ivar+sys+upDown, None )

            # Create total uncertainty and sys uncertainty plots.
            uncerUnfoldHisto2[ivar+sys+'Total'] = unfoldHisto2.Clone("unfoldHisto2SysUnc")          # Syst uncertainty
            uncerUnfoldHisto2[ivar+sys+'Total'].Reset()
            uncerUnfoldHisto2[ivar+sys+'Total'].SetLineStyle(3)
            for i in range( 0, unfoldHisto2.GetNbinsX() + 1):
                try: yup = abs( uncerUnfoldHisto2[ivar+sys+'Up'].GetBinContent(i))
                except KeyError: yup = 0
                try: ydn = abs( uncerUnfoldHisto2[ivar+sys+'Down'].GetBinContent(i))
                except KeyError: ydn = 0
                dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
                uncerUnfoldHisto2[ivar+sys+'Total'].SetBinContent(i, dy )
            unfoldHisto2SysUnc.Add( uncerUnfoldHisto2[ivar+sys+'Total'] )

    ###### Plot unfolding results
    #tdrStyle.SetPadRightMargin(0.05)
    #tdrStyle.SetPadLeftMargin(0.15)
    can_2_ = ROOT.TCanvas('can_2_', 'can_2_',  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()
    #ROOT.gStyle.SetOptStat(0)

    pad1.cd()
    
    #tmpPad1 = pad1.DrawFrame( 0, 0.3, 1, 1.7 )
    
    #tmpPad1.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(0.5)})}' )
    #tmpPad1.GetYaxis().SetTitleOffset(0.9)
    #tmpPad1.GetYaxis().SetTitleSize(28)
    #tmpPad1.GetYaxis().SetLabelSize(20)
    #tmpPad1.SetStats(0)
    
    leg=ROOT.TLegend(0.25,0.6,0.55,0.88)
    leg.SetLineColor(0)
    leg.SetLineWidth(0)
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    #leg.SetTextSize(0.8)

    #genJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_genJet'+ivar+sel ].Clone()
    genJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_genJet'+ivar+sel+'_Normalized' ].Clone()
    genJetHisto2.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(0.5)})}' )
    genJetHisto2.GetYaxis().SetTitleSize( 0.05 )
    genJetHisto2.GetYaxis().SetTitleOffset( 0.9 )
    #genJetHisto2.GetYaxis().CenterTitle()
    genJetHisto2.SetStats(0)
    genJetHisto2.Scale(1, 'width')  ### divide by bin width
    genJetHisto2.SetLineWidth(2)
    genJetHisto2.GetYaxis().SetLabelSize(0.037)
    genJetHisto2.SetLineColor(1)
    pad1.Modified()
    pad1.Update()

    leg.AddEntry( genJetHisto2, 'Gen-level MC', 'l' )

    #unfoldHisto2 = tunfolder2.GetOutput("unfolded")
    #if len(variables[ivar])>1: unfoldHisto2 = unfoldHisto2.Rebin( len(variables[ivar])-1, unfoldHisto2.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    #else: unfoldHisto2.Rebin( variables[ivar][0] )
    

    unfoldHisto2woUnc.Scale(1, 'width')  ### divide by bin width
    unfoldHisto2woUnc.SetMarkerStyle(0)
    unfoldHisto2woUnc.SetMarkerColor(ROOT.kRed-5)
    unfoldHisto2woUnc.SetLineColor(ROOT.kRed-5)
    unfoldHisto2woUnc.SetLineWidth(2)
    unfoldHisto2woUnc.SetStats(0)
    leg.AddEntry( unfoldHisto2woUnc, 'Unfolded, stat+unf unc', 'l' )
    
    unfoldHisto2.Scale(1, 'width')  ### divide by bin width
    unfoldHisto2.SetMarkerStyle(4)
    unfoldHisto2.SetMarkerSize(2)
    unfoldHisto2.SetMarkerColor(ROOT.kRed+1)
    unfoldHisto2.SetLineColor(ROOT.kRed+1)
    unfoldHisto2.SetStats(0)
    leg.AddEntry( unfoldHisto2, 'Unfolded, total unc', 'pl' )

    foldHisto2 = tunfolder2.GetFoldedOutput("folded") #, 'folded', 'folded', , False)
    if len(variables[ivar])>1: foldHisto2 = foldHisto2.Rebin( len(variables[ivar])-1, foldHisto2.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    else: foldHisto2.Rebin( 2 )
    foldHisto2.Scale(1, 'width')  ### divide by bin width
    foldHisto2.SetLineWidth(2)
    foldHisto2.SetLineStyle(2)
    foldHisto2.SetLineColor(8)
    foldHisto2.SetStats(0)
    leg.AddEntry( foldHisto2, 'Folded', 'l' )

    if process.startswith('data'):
        #recoJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel+'_Normalized' ].Clone()
        recoJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
        recoJetHisto2.Scale(1/recoJetHisto2.Integral(), 'width')  ### divide by bin width
        recoJetHisto2.SetLineWidth(2)
        recoJetHisto2.SetLineStyle(2)
        recoJetHisto2.SetLineColor(ROOT.kBlue)
        leg.AddEntry( recoJetHisto2, 'Reco-level MC', 'l' )

    dataJetHisto2 = dataMinusBkgs2GenBin.Clone()
    dataJetHisto2.Scale(1/dataJetHisto2.Integral(), 'width')  ### divide by bin width
    dataJetHisto2.SetLineWidth(2)
    dataJetHisto2.SetLineStyle(2)
    dataJetHisto2.SetLineColor(ROOT.kMagenta)
    dataJetHisto2.SetStats(0)
    leg.AddEntry( dataJetHisto2, ('Data' if process.startswith('data') else 'MC Closure' ), 'l' )

    
    
    genJetHisto2.SetMaximum( 1.3*max([ genJetHisto2.GetMaximum(), unfoldHisto2.GetMaximum(), unfoldHisto2woUnc.GetMaximum(), dataJetHisto2.GetMaximum() ] )  )
    pad1.Update()
    pad1.Modified()
    
    genJetHisto2.Draw( "histe")
    unfoldHisto2.Draw( "same")
    unfoldHisto2woUnc.Draw( "e1 same")
    foldHisto2.Draw( "hist same")
    dataJetHisto2.Draw( "hist same")
    if process.startswith('data'): recoJetHisto2.Draw( "hist same")
    #ROOT.gStyle.SetOptStat(0)
    leg.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    #ROOT.gPad.SetLogy()
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.relPosX = 0.13
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    ROOT.gStyle.SetOptStat(0000000)
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    #tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1. )
    
    h2RatioUp = genJetHisto2.Clone()
    h2RatioUp.GetYaxis().SetTitle( "#frac{MC-Truth}{Unfolded}" )
    h2RatioUp.GetYaxis().SetTitleOffset( 0.6)
    h2RatioUp.GetYaxis().SetTitleSize(0.1)
    h2RatioUp.GetYaxis().SetLabelSize(0.085)
    h2RatioUp.GetXaxis().SetTitle("#tau_{1}^{(0.5)}")
    h2RatioUp.GetXaxis().SetTitleOffset( 0.8)
    h2RatioUp.GetXaxis().SetTitleSize( 0.15)
    h2RatioUp.GetXaxis().SetLabelSize(0.085)
    h2RatioUp.GetYaxis().CenterTitle()
    h2RatioUp.Divide( unfoldHisto2 )
    h2RatioUp.GetYaxis().SetRangeUser(0.,2.)
    h2RatioUp.Draw('P0')
    
    #tmpPad2.SetLabelSize(16, 'x')
    #tmpPad2.SetTitleSize(19, 'x')
    #tmpPad2.SetLabelSize(16., 'y')
    #tmpPad2.SetTitleSize(19., 'y')
    #tmpPad2.SetNdivisions(505, 'x')
    #tmpPad2.SetNdivisions(505, 'y')
    #tmpPad2.SetMarkerStyle(8)
    #h2RatioUp.SetTitle("#tau_{1}^{(0.5)}")
    #h2RatioUp.SetLabelSize(18)
    
    #h2RatioUp.GetXaxis().SetTitle( genJetHisto2.GetXaxis().GetTitle() )
    pad2.Modified()

    #pad2.Update()
    #pad2.Modified()
    #h2RatioDown.Draw('P same')
    #pad1.cd()
    #leg.Draw()
    #can_2_.Update()
    #can_2_.Modified()
    can_2_.Draw()
    can_2_.SaveAs('Plots/'+'MG_'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_'+version+'.png')

    can_2_Unc = ROOT.TCanvas('can_2_Unc', 'can_2_Unc',  10, 10, 750, 500 )
    can_2_Unc.SetTitle("")
    #can_2_Unc.SetLogy()
    #ROOT.gStyle.SetOptStat(0)
    legend=ROOT.TLegend(0.25,0.65,0.35,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)
    legend.SetLineStyle(0)
    legend.AddEntry( unfoldHisto2TotUnc, 'Total Unc', 'l' )

    unfoldHisto2TotUnc.SetStats(0)
    unfoldHisto2TotUnc.SetLineWidth(2)
    unfoldHisto2TotUnc.Scale( 1/(unfoldHisto2TotUnc.Integral()) )
    unfoldHisto2TotUnc.SetMaximum(1.)
    unfoldHisto2TotUnc.SetMinimum(0.000001)
    unfoldHisto2TotUnc.GetYaxis().SetTitle('Fractional Uncertainty')
    unfoldHisto2TotUnc.Draw('hist')

    if len(sysUncert)>0 :
        legend.AddEntry( unfoldHisto2SysUnc, 'Total Syst Unc', 'l' )
        unfoldHisto2SysUnc.Scale( 1/unfoldHisto2TotUnc.Integral() )
        unfoldHisto2SysUnc.SetLineWidth(2)
        unfoldHisto2SysUnc.SetStats(0)
        unfoldHisto2SysUnc.Draw("hist same")
        dummy=2
        for k in uncerUnfoldHisto2:
            if k.endswith('Total'):
                for i in sysUncert:
                    if i in k:
                        legend.AddEntry( uncerUnfoldHisto2[k], i.split('_')[1]+" Total", 'l' )
                uncerUnfoldHisto2[k].SetLineColor(dummy)
                uncerUnfoldHisto2[k].SetLineWidth(2)
                uncerUnfoldHisto2[k].SetStats(0)
                uncerUnfoldHisto2[k].Scale( 1/unfoldHisto2TotUnc.Integral() )
                uncerUnfoldHisto2[k].Draw("hist same")
                dummy=dummy+1

    legend.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    ROOT.gPad.SetLogy()
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can_2_Unc, 4, 0)
    can_2_Unc.Draw()
    can_2_Unc.SaveAs('Plots/'+'MG_'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_UNC_'+version+'.png')


|------> Unfolding _tau_0p5_0
|------> Cross check: plotting data vs all MC
Processing....... _tau_0p5_0_fromData_TTJets_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png
Processing....... _tau_0p5_0_fromData_TTJets_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png
|------> Cross check: plotting response matrix for signal
35.92 fb^{-1}, 2016 (13 TeV)
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding starts:
|------> TUnfolding adding input:
|------> TUnfolding adding uncert:
_jesTotalUp
35.92 fb^{-1}, 2016 (13 TeV)
_jesTotalDown
35.92 fb^{-1}, 2016 (13 TeV)
_jerUp
35.92 fb^{-1}, 2016 (13 TeV)
_jerDown
35.92 fb^{-1}, 2016 (13 TeV)
_puUp
35.92 fb^{-1}, 2016 (13 TeV)
_puDown
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
35.92 fb^{-1},  2016 (13 TeV)
35.92 fb^{-1},  2016 (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_fromData_TTJets_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_fromData_TTJets_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/MG__tau_0p5_0_fromData_TTJets_topSel_responseMatrixv01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/MG__tau_0p5_0_fromData_TTJets_topSelNormalized_responseMatrixv01_fin.png has been created
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 5 output bins do not depend on the input data  0-1 7-9
Info in <TUnfold::TUnfold>: 16 input bins and 5 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:[ufl])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfoldin

In [ ]:
###### Plot unfolding results
#tdrStyle.SetPadRightMargin(0.05)
#tdrStyle.SetPadLeftMargin(0.15)
can_combined = ROOT.TCanvas('can_combined', 'can_combined',  10, 10, 750, 750 )
pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
pad1.Draw()
pad2.Draw()
ROOT.gStyle.SetOptStat(0)

pad1.cd()

leg=ROOT.TLegend(0.25,0.6,0.55,0.88)
leg.SetLineColor(0)
leg.SetLineWidth(0)
leg.SetBorderSize(0)
leg.SetFillStyle(0)
#leg.SetTextSize(0.8)

#genJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_genJet'+ivar+sel ].Clone()
genJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_genJet'+ivar+sel+'_Normalized' ].Clone()
genJetHisto2.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(0.5)})}' )
genJetHisto2.GetYaxis().SetTitleSize( 0.05 )
genJetHisto2.GetYaxis().SetTitleOffset( 0.9 )
#genJetHisto2.GetYaxis().CenterTitle()
genJetHisto2.SetStats(0)
#genJetHisto2.Scale(1, 'width')  ### divide by bin width
genJetHisto2.SetLineWidth(2)
genJetHisto2.GetYaxis().SetLabelSize(0.037)
genJetHisto2.SetLineColor(rt.kAzure+8)
genJetHisto1.SetLineColor(rt.kRed+2)


pad1.Modified()
pad1.Update()

leg.AddEntry( genJetHisto2, 'Gen-level MC (MadGraph MLM)', 'l' )
leg.AddEntry( genJetHisto1, 'Gen-level MC (Pythia8)', 'l')

#unfoldHisto2 = tunfolder2.GetOutput("unfolded")
#if len(variables[ivar])>1: unfoldHisto2 = unfoldHisto2.Rebin( len(variables[ivar])-1, unfoldHisto2.GetName()+"_rebin", array( 'd', variables[ivar] ) )
#else: unfoldHisto2.Rebin( variables[ivar][0] )


unfoldHisto2woUnc.Scale(1, 'width')  ### divide by bin width
unfoldHisto2woUnc.SetMarkerStyle(0)
unfoldHisto2woUnc.SetMarkerColor(ROOT.kRed-5)
unfoldHisto2woUnc.SetLineColor(ROOT.kRed-5)
unfoldHisto2woUnc.SetLineWidth(2)
unfoldHisto2woUnc.SetStats(0)
leg.AddEntry( unfoldHisto2woUnc, 'Unfolded, stat+unf unc', 'l' )

unfoldHisto2.Scale(1, 'width')  ### divide by bin width
unfoldHisto2.SetMarkerStyle(4)
unfoldHisto2.SetMarkerSize(2)
unfoldHisto2.SetMarkerColor(ROOT.kRed+1)
unfoldHisto2.SetLineColor(ROOT.kRed+1)
unfoldHisto2.SetStats(0)
leg.AddEntry( unfoldHisto2, 'Unfolded, total unc', 'pl' )

foldHisto2 = tunfolder2.GetFoldedOutput("folded") #, 'folded', 'folded', , False)
if len(variables[ivar])>1: foldHisto2 = foldHisto2.Rebin( len(variables[ivar])-1, foldHisto2.GetName()+"_rebin", array( 'd', variables[ivar] ) )
else: foldHisto2.Rebin( 2 )
foldHisto2.Scale(1, 'width')  ### divide by bin width
foldHisto2.SetLineWidth(2)
foldHisto2.SetLineStyle(2)
foldHisto2.SetLineColor(8)
foldHisto2.SetStats(0)
leg.AddEntry( foldHisto2, 'Folded', 'l' )

if process.startswith('data'):
    #recoJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel+'_Normalized' ].Clone()
    recoJetHisto2 = signalHistos2[ next(iter(altsigFiles))+'_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
    recoJetHisto2.Scale(1/recoJetHisto2.Integral(), 'width')  ### divide by bin width
    recoJetHisto2.SetLineWidth(2)
    recoJetHisto2.SetLineStyle(2)
    recoJetHisto2.SetLineColor(ROOT.kBlue)
    leg.AddEntry( recoJetHisto2, 'Reco-level MC', 'l' )

dataJetHisto2 = dataMinusBkgs2GenBin.Clone()
dataJetHisto2.Scale(1/dataJetHisto2.Integral(), 'width')  ### divide by bin width
dataJetHisto2.SetLineWidth(2)
dataJetHisto2.SetLineStyle(2)
dataJetHisto2.SetLineColor(ROOT.kMagenta)
dataJetHisto2.SetStats(0)
leg.AddEntry( dataJetHisto2, ('Data' if process.startswith('data') else 'MC Closure' ), 'l' )



genJetHisto2.SetMaximum( 1.3*max([ genJetHisto2.GetMaximum(), unfoldHisto2.GetMaximum(), unfoldHisto2woUnc.GetMaximum(), dataJetHisto2.GetMaximum() ] )  )
pad1.Update()
pad1.Modified()

genJetHisto2.Draw( "histe")
unfoldHisto2.Draw( "same")
unfoldHisto2woUnc.Draw( "e1 same")
foldHisto2.Draw( "hist same")
dataJetHisto2.Draw( "hist same")
if process.startswith('data'): recoJetHisto2.Draw( "hist same")
#ROOT.gStyle.SetOptStat(0)
leg.Draw()
CMS_lumi.extraText = "Preliminary"
CMS_lumi.cmsTextSize = 1.2
CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
#ROOT.gPad.SetLogy()
CMS_lumi.cmsTextSize = 1.2
CMS_lumi.relPosX = 0.13
CMS_lumi.CMS_lumi(pad1, 4, 0)

pad2.cd()
ROOT.gStyle.SetOptFit(1)
ROOT.gStyle.SetOptStat(0000000)
pad2.SetGrid()
pad2.SetTopMargin(0)
pad2.SetBottomMargin(0.3)
#tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1. )

h2RatioUp = genJetHisto2.Clone()
h2RatioUp.GetYaxis().SetTitle( "#frac{MC-Truth}{Unfolded}" )
h2RatioUp.GetYaxis().SetTitleOffset( 0.6)
h2RatioUp.GetYaxis().SetTitleSize(0.1)
h2RatioUp.GetYaxis().SetLabelSize(0.085)
h2RatioUp.GetXaxis().SetTitle("#tau_{1}^{(0.5)}")
h2RatioUp.GetXaxis().SetTitleOffset( 0.8)
h2RatioUp.GetXaxis().SetTitleSize( 0.15)
h2RatioUp.GetXaxis().SetLabelSize(0.085)
h2RatioUp.GetYaxis().CenterTitle()
h2RatioUp.Divide( unfoldHisto2 )
h2RatioUp.GetYaxis().SetRangeUser(0.,2.)
h2RatioUp.Draw('P0')

#h2RatioUp.GetXaxis().SetTitle( genJetHisto2.GetXaxis().GetTitle() )
pad2.Modified()

#pad2.Update()
#pad2.Modified()
#h2RatioDown.Draw('P same')
#pad1.cd()
#leg.Draw()
#can_combined.Update()
#can_combined.Modified()
can_combined.Draw()
can_combined.SaveAs('Plots/'+'MG_'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_'+version+'.png')


In [ ]:
print variables

In [31]:
h=0
for i in signalHistos.keys():
    #print i
    if 'resp' in i and 'nom' in i and not 'Norm' in i:
        print (i)
        h = signalHistos[i]
        
pur, stab = purity_stability(h0=h, bins=np.array(variables[variables.keys()[0]]))

TT_respJet_tau_0p5_0_nom_topSel


/shome/kdatta/.local/lib/python2.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in divide
/shome/kdatta/.local/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in divide


In [32]:
print pur, stab

Name: Purity Title:  NbinsX: 8 Name: Stability Title:  NbinsX: 8


In [33]:
can_ps = ROOT.TCanvas('can_ps', 'can_ps',  10, 10, 750, 500 )
can_ps.SetTitle("")
#can_ps.SetLogy()
#ROOT.gStyle.SetOptStat(0)
legend=ROOT.TLegend(0.25,0.65,0.45,0.88)
legend.SetFillStyle(0)
legend.SetTextSize(0.03)
legend.SetBorderSize(0)
legend.SetLineStyle(0)
legend.AddEntry( pur, 'Purity', 'l' )

pur.SetStats(0)
pur.SetLineWidth(2)
pur.SetLineColor(ROOT.kRed-2)
pur.SetLineStyle(1)
pur.GetYaxis().SetTitle('Purity/Stability')
pur.GetXaxis().SetTitle('#tau_{1}^{(0.5)}')
pur.Draw('hist')
legend.AddEntry( stab, 'Stability', 'l' )

stab.SetStats(0)
stab.SetLineWidth(2)
stab.SetLineColor(ROOT.kBlue+2)
stab.SetLineStyle(2)
stab.Draw('hist same')

legend.Draw()
CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
#ROOT.gPad.SetLogy()
CMS_lumi.relPosX = 0.11
CMS_lumi.CMS_lumi(can_ps, 4, 0)
#can_ps.Draw()
can_ps.SaveAs('Plots/'+variables.keys()[0]+sel+'_from'+'_PurityandStability_'+version+'.png')

35.92 fb^{-1},  2016 (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: can_ps
Info in <TCanvas::Print>: png file Plots/_tau_0p5_0_topSel_from_PurityandStability_v01_fin.png has been created


In [34]:
variables = {}
#variables[ '_tau_0p5_0' ] = [10] #gen_bins2  ### (reco,gen)
variables[ '_tau_1_0' ] = array('d',[0., 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.5, 0.56, 0.62, 0.8])#[0. 0.25 0.30625 0.3625 0.418750.475 0.53125 0.5875 0.64375 0.7 0.8 ][0., 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.65]#[ 6]
#variables[ '_tau_2_0' ] = [ 3]
#variables[ '_tau_0p5_1' ] = [10] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_1' ] = [ 8]
#variables[ '_tau_2_1' ] = [ 5]
#variables[ '_tau_0p5_2' ] = [8] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_2' ] = [ 5]
#variables[ '_tau_2_2' ] = [ 4]

#variables[ '_tau_0p5_3' ] = [8] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_3' ] = [ 5]
#variables[ '_tau_2_3' ] = [0., 0.006, 0.012, 0.018, 0.024, 0.03, 0.04, 0.055, 0.075, 0.11] #[4]#[0., 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02, 0.0225, 0.025]

sysUncert = [ '_jesTotal', '_jer', '_pu' ]
print variables

{'_tau_1_0': array('d', [0.0, 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.5, 0.56, 0.62, 0.8])}


In [35]:
sel= '_topSel'
process = 'data'
version = 'v01_fin'

In [36]:
dataFile = {}
dataFile['data'] = [ TFile('Histos_SysUnc/jetObservables_histograms_SingleMuonRun2016ALL.root' ), 'Data', 'kBlack' ]

In [37]:
sigFiles = {}
sigFiles['TT'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TT_TuneCUETP8M2T4_13TeV-powheg-pythia8.root'), 'ttbar (powheg)', 'kRed' ]
#sigFiles['TTJets'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8.root'), 'ttbar (MG)', 'kBlue' ]

In [38]:
bkgFiles = {}
bkgFiles['ST_s-channel_4f_InclusiveDecays'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_s-channel_4f_InclusiveDecays_13TeV-amcatnlo-pythia8.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_antitop'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_t-channel_antitop_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_top'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_t-channel_top_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_antitop'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M2T4.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_top'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M2T4.root' ), 'Single top', 'kMagenta' ]
#bkgFiles['WJets'] = [ TFile( inputFolder+'/jetObservables_histograms_WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8.root' ), 'WJets', 'kCyan' ]
###bkgFiles[] = [ '', TFile( inputFolder+'/jetObservables_histograms_'+ibkg+'.root' ), '', 'kMagenta' ]

In [39]:
dataHistos = loadHistograms( dataFile, variables, sel='_topSel', isMC=False )
signalHistos = loadHistograms( sigFiles, variables, sel='_topSel', sysUnc=sysUncert )
bkgHistos = loadHistograms( bkgFiles, variables, sel='_topSel', sysUnc=sysUncert ) 

('New variable bins:', array('d', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8]), 'recoJet_tau_1_0_nom_topSel', array('d', [0.0, 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.5, 0.56, 0.62, 0.8]))
('New variable bins:', array('d', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8]), 'recoJet_tau_1_0_nom_topSel', array('d', [0.0, 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.5, 0.56, 0.62, 0.8]))
('New variable bins:', array('d', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8]), 'recoJet_tau_1_0_jesTotalDown_topSel', array('d', [0.0, 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.5, 0.56, 0.62, 0.8]))
('New variable bins:', array('d', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.5

('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH

('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])
('New variable bins TH2F:', [0.0, 0.05, 0.1, 0.13, 0.16, 0.19, 0.22, 0.25, 0.28, 0.31, 0.34, 0.37, 0.4, 0.43, 0.46, 0.48, 0.5, 0.53, 0.56, 0.59, 0.62, 0.71, 0.8])


In [40]:
for ivar in variables:
    
    print '|------> Unfolding '+ivar

    ######## Cross check: plotting data vs all MC Scaled
    print '|------> Cross check: plotting data vs all MC'
    allBkgHisto = dataHistos['data_recoJet'+ivar+'_nom'+sel].Clone()
    allBkgHisto.Reset()
    allBkgHistoNorm = allBkgHisto.Clone()
    allBkgHistoGenBin = dataHistos['data_recoJet'+ivar+'_nom'+sel+'_genBin'].Clone()
    allBkgHistoGenBin.Reset()
    for ibkg in bkgHistos:
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel): allBkgHisto.Add( bkgHistos[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_genBin'): allBkgHistoGenBin.Add( bkgHistos[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'): allBkgHistoNorm.Add( bkgHistos[ibkg].Clone() )
    allMCHisto = allBkgHisto.Clone()
    allMCHisto.Add( signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel ].Clone() )
    plotSimpleComparison( dataHistos['data_recoJet'+ivar+'_nom'+sel].Clone(), 'data', allMCHisto, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom", rebinX=variables[ivar][0], version=sel+'_'+version  )
    allMCHistoNorm = allBkgHistoNorm.Clone()
    allMCHistoNorm.Add( signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized" ].Clone() )
    plotSimpleComparison( dataHistos['data_recoJet'+ivar+'_nom'+sel+"_Normalized"].Clone(), 'data', allMCHistoNorm, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom_Normalized", rebinX=variables[ivar][0], version=sel+'_'+version  )

    ######## Cross check: plotting response matrix
    #tdrStyle.SetPadRightMargin(0.12)
    print '|------> Cross check: plotting response matrix for signal'
    can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750,600 )
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].SetStats(0)
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].Draw("colz")
    can2D.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can2D, 4, 0)
    can2D.Draw()
    can2D.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'_responseMatrix'+version+'.png')
    can2DNorm = ROOT.TCanvas(ivar+'can2DNorm', ivar+'can2DNorm', 750, 600 )
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].SetStats(0)
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].Draw("colz")
    can2DNorm.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can2DNorm, 4, 0)
    can2DNorm.Draw()
    can2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'Normalized_responseMatrix'+version+'.png')

    ####### Removing bkgs from data
    dataMinusBkgs = dataHistos[ 'data_recoJet'+ivar+'_nom'+sel ].Clone()
    dataMinusBkgs.Add( allBkgHisto.Clone(), -1 )
    dataMinusBkgs.Scale( 1/dataMinusBkgs.Integral() )
    dataMinusBkgsGenBin = dataHistos[ 'data_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
    dataMinusBkgsGenBin.Add( allBkgHistoGenBin.Clone(), -1 )
    dataMinusBkgsGenBin.Scale( 1/dataMinusBkgsGenBin.Integral() )

    ######## TUnfold part
    print '|------> TUnfolding starts:'

    ##### Defining options for TUnfold
    tunfolder = ROOT.TUnfoldDensity(
                                        signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'], ### response matrix
                                        #signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel], ### response matrix
                                        ROOT.TUnfold.kHistMapOutputHoriz,  #### kHistMapOutputVert if x->reco and y->gen, kHistMapOutputHoriz if x->gen and y->reco
                                        ROOT.TUnfold.kRegModeCurvature,   ##### Regularization Mode : ROOT.TUnfold.kRegModeCurvature regularizes based on the 2nd derivative of the output. More information wrt the other options can be gained from reading the source code
                                        ROOT.TUnfold.kEConstraintNone,    ##### Constraint : TUnfold.kEConstraintNone meaning we do not constrain further, the other option is to force constraint of area. (Need to look into this!!)
                                        ROOT.TUnfoldDensity.kDensityModeBinWidth  ##### Density Mode: ROOT.TUnfoldDensity.kDensityModeBinWidth uses the bin width to normalize the event rate in a given bin, accounting for non-uniformity in bin widths as discussed in section 7.2.1 of the TUnfold paper
                                        )

    ##### Defining input (data recoJet )
    print '|------> TUnfolding adding input:'
    #tunfolder.SetInput( dataHistos[ 'data_recoJet'+ivar+'_nom'+sel ].Clone() )
    tunfolder.SetInput( dataMinusBkgs )

    ###### Removing bkgs from data using TUnfold. Better to subtract bkgs beforehand
    #for ibkg in bkgHistos:
    #    if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'):
    #        print '|--------> Removing this bkg: ', ibkg
    #        tunfolder.SubtractBackground( bkgHistos[ibkg], ibkg )

    ###### Adding SYS unc
    if len(sysUncert)>0 :
        print '|------> TUnfolding adding uncert:'
        for sys in sysUncert:
            #plotSysComparison( signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized"],
            #                    signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Up'+sel+"_Normalized"],
            #                    signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Down'+sel+"_Normalized"],
            #                    ivar+'_'+next(iter(sigFiles)),
            #                    sys.split('_')[1],
            #                    version=sel+'_'+version
            #                    )
            for upDown in [ 'Up', 'Down' ]:
                print sys+upDown
                tunfolder.AddSysError(
                                    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"],
                                    sys+upDown,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                    )
                can2DNorm = ROOT.TCanvas(ivar+'can2DNorm'+sys+upDown, ivar+'can2DNorm'+sys+upDown, 750, 600 )
                signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].SetStats(0)
                signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].Draw("colz")
                can2DNorm.SetRightMargin(0.13)
                CMS_lumi.extraText = "  Preliminary"
                CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
                #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
                CMS_lumi.relPosX = 0.11
                CMS_lumi.CMS_lumi(can2DNorm, 4, 0)
                can2DNorm.Draw()
                can2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+sys+upDown+'Normalized_responseMatrix'+version+'.png')

    ###### Running the unfolding
    print '|------> TUnfolding doUnfold:'
    tunfolder.DoUnfold(0)

    ###### Regularization
#        nScan=50
#        tauMin=0.0
#        tauMax=0.0
#        iBest=0
#
#        logTauX = ROOT.MakeNullPointer(ROOT.TSpline)
#        logTauY = ROOT.MakeNullPointer(ROOT.TSpline)
#        lCurve = ROOT.MakeNullPointer(ROOT.ROOT.TGraph)
#        ## this method scans the parameter tau and finds the kink in the L curve finally, the unfolding is done for the best choice of tau
#        tunfolder.ScanLcurve(nScan,tauMin,tauMax,lCurve,logTauX,logTauY)
    #########################
    ROOT.gStyle.SetOptStat(0)

    ##### Get output of unfolding
    unfoldHisto = tunfolder.GetOutput("unfoldHisto")

    #### Get various covariances
    print '|------> TUnfolding covariances'
    cov = tunfolder.GetEmatrixTotal("cov", "Covariance Matrix")
    cov_uncorr = tunfolder.GetEmatrixSysUncorr("cov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties")
    cov_uncorr_data = tunfolder.GetEmatrixInput("cov_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data")
    unfoldHistowoUnc = unfoldHisto.Clone()        # Unfolding and stat unc
    unfoldHistoStatUnc = unfoldHisto.Clone("unfoldHistoStatUnc")        # Unfolding and stat unc
    unfoldHistoTotUnc = unfoldHisto.Clone("unfoldHistoTotUnc")          # Total uncertainty
    for ibin in range( 0, unfoldHisto.GetNbinsX()+1 ):
        unc_tot = ROOT.TMath.Sqrt( cov.GetBinContent(ibin,ibin) )
        unfoldHistoTotUnc.SetBinContent(ibin, unc_tot )
        unfoldHisto.SetBinError(ibin, unc_tot )


    ##### Get systematic shifts of output
    uncerUnfoldHisto = {}
    if len(sysUncert)>0 :
        print '|------> TUnfolding uncertainties:'
        unfoldHistoSysUnc = unfoldHisto.Clone("unfoldHistoSysUnc")          # Syst uncertainty
        unfoldHistoSysUnc.Reset()
        unfoldHistoSysUnc.SetLineStyle(2)

        for sys in sysUncert:
            for upDown in [ 'Up', 'Down' ]:
                print sys+upDown
                uncerUnfoldHisto[ivar+sys+upDown] = tunfolder.GetDeltaSysSource(sys+upDown, "unfoldHisto_"+sys+upDown+"shift", "-1#sigma")
                try: uncerUnfoldHisto[ivar+sys+upDown].SetLineStyle(2)
                except ReferenceError: uncerUnfoldHisto.pop( ivar+sys+upDown, None )

            # Create total uncertainty and sys uncertainty plots.
            uncerUnfoldHisto[ivar+sys+'Total'] = unfoldHisto.Clone("unfoldHistoSysUnc")          # Syst uncertainty
            uncerUnfoldHisto[ivar+sys+'Total'].Reset()
            uncerUnfoldHisto[ivar+sys+'Total'].SetLineStyle(3)
            for i in xrange( 0, unfoldHisto.GetNbinsX() + 1):
                try: yup = abs( uncerUnfoldHisto[ivar+sys+'Up'].GetBinContent(i))
                except KeyError: yup = 0
                try: ydn = abs( uncerUnfoldHisto[ivar+sys+'Down'].GetBinContent(i))
                except KeyError: ydn = 0
                dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
                uncerUnfoldHisto[ivar+sys+'Total'].SetBinContent(i, dy )
            unfoldHistoSysUnc.Add( uncerUnfoldHisto[ivar+sys+'Total'] )

    ###### Plot unfolding results
    #tdrStyle.SetPadRightMargin(0.05)
    #tdrStyle.SetPadLeftMargin(0.15)
    can = ROOT.TCanvas('can', 'can',  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()
    #ROOT.gStyle.SetOptStat(0)

    pad1.cd()
    
    #tmpPad1 = pad1.DrawFrame( 0, 0.3, 1, 1.7 )
    
    #tmpPad1.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(1)})}' )
    #tmpPad1.GetYaxis().SetTitleOffset(0.9)
    #tmpPad1.GetYaxis().SetTitleSize(28)
    #tmpPad1.GetYaxis().SetLabelSize(20)
    #tmpPad1.SetStats(0)
    
    leg=ROOT.TLegend(0.65,0.6,0.95,0.88)
    leg.SetLineColor(0)
    leg.SetLineWidth(0)
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    #leg.SetTextSize(0.8)

    #genJetHisto = signalHistos[ next(iter(sigFiles))+'_genJet'+ivar+sel ].Clone()
    genJetHisto = signalHistos[ next(iter(sigFiles))+'_genJet'+ivar+sel+'_Normalized' ].Clone()
    genJetHisto.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(1)})}' )
    genJetHisto.GetYaxis().SetTitleSize( 0.05 )
    genJetHisto.GetYaxis().SetTitleOffset( 0.9 )
    #genJetHisto.GetYaxis().CenterTitle()
    genJetHisto.SetStats(0)
    genJetHisto.Scale(1, 'width')  ### divide by bin width
    genJetHisto.SetLineWidth(2)
    genJetHisto.GetYaxis().SetLabelSize(0.037)
    genJetHisto.SetLineColor(1)
    pad1.Modified()
    pad1.Update()

    leg.AddEntry( genJetHisto, 'Gen-level MC', 'l' )

    #unfoldHisto = tunfolder.GetOutput("unfolded")
    #if len(variables[ivar])>1: unfoldHisto = unfoldHisto.Rebin( len(variables[ivar])-1, unfoldHisto.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    #else: unfoldHisto.Rebin( variables[ivar][0] )
    

    unfoldHistowoUnc.Scale(1, 'width')  ### divide by bin width
    unfoldHistowoUnc.SetMarkerStyle(0)
    unfoldHistowoUnc.SetMarkerColor(ROOT.kRed-5)
    unfoldHistowoUnc.SetLineColor(ROOT.kRed-5)
    unfoldHistowoUnc.SetLineWidth(2)
    unfoldHistowoUnc.SetStats(0)
    leg.AddEntry( unfoldHistowoUnc, 'Unfolded, stat+unf unc', 'l' )
    
    unfoldHisto.Scale(1, 'width')  ### divide by bin width
    unfoldHisto.SetMarkerStyle(4)
    unfoldHisto.SetMarkerSize(2)
    unfoldHisto.SetMarkerColor(ROOT.kRed+1)
    unfoldHisto.SetLineColor(ROOT.kRed+1)
    unfoldHisto.SetStats(0)
    leg.AddEntry( unfoldHisto, 'Unfolded, total unc', 'pl' )

    foldHisto = tunfolder.GetFoldedOutput("folded") #, 'folded', 'folded', , False)
    if len(variables[ivar])>1: foldHisto = foldHisto.Rebin( len(variables[ivar])-1, foldHisto.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    else: foldHisto.Rebin( 2 )
    foldHisto.Scale(1, 'width')  ### divide by bin width
    foldHisto.SetLineWidth(2)
    foldHisto.SetLineStyle(2)
    foldHisto.SetLineColor(8)
    foldHisto.SetStats(0)
    leg.AddEntry( foldHisto, 'Folded', 'l' )

    if process.startswith('data'):
        #recoJetHisto = signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_Normalized' ].Clone()
        recoJetHisto = signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
        recoJetHisto.Scale(1/recoJetHisto.Integral(), 'width')  ### divide by bin width
        recoJetHisto.SetLineWidth(2)
        recoJetHisto.SetLineStyle(2)
        recoJetHisto.SetLineColor(ROOT.kBlue)
        leg.AddEntry( recoJetHisto, 'Reco-level MC', 'l' )

    dataJetHisto = dataMinusBkgsGenBin.Clone()
    dataJetHisto.Scale(1/dataJetHisto.Integral(), 'width')  ### divide by bin width
    dataJetHisto.SetLineWidth(2)
    dataJetHisto.SetLineStyle(2)
    dataJetHisto.SetLineColor(ROOT.kMagenta)
    dataJetHisto.SetStats(0)
    leg.AddEntry( dataJetHisto, ('Data' if process.startswith('data') else 'MC Closure' ), 'l' )

    
    
    genJetHisto.SetMaximum( 1.3*max([ genJetHisto.GetMaximum(), unfoldHisto.GetMaximum(), unfoldHistowoUnc.GetMaximum(), dataJetHisto.GetMaximum() ] )  )
    pad1.Update()
    pad1.Modified()
    
    genJetHisto.Draw( "histe")
    unfoldHisto.Draw( "same")
    unfoldHistowoUnc.Draw( "e1 same")
    foldHisto.Draw( "hist same")
    dataJetHisto.Draw( "hist same")
    if process.startswith('data'): recoJetHisto.Draw( "hist same")
    #ROOT.gStyle.SetOptStat(0)
    leg.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    #ROOT.gPad.SetLogy()
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.relPosX = 0.13
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    ROOT.gStyle.SetOptStat(0000000)
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    #tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1. )
    
    hRatioUp = genJetHisto.Clone()
    hRatioUp.GetYaxis().SetTitle( "#frac{MC-Truth}{Unfolded}" )
    hRatioUp.GetYaxis().SetTitleOffset( 0.6)
    hRatioUp.GetYaxis().SetTitleSize(0.1)
    hRatioUp.GetYaxis().SetLabelSize(0.085)
    hRatioUp.GetXaxis().SetTitle("#tau_{1}^{(1)}")
    hRatioUp.GetXaxis().SetTitleOffset( 0.8)
    hRatioUp.GetXaxis().SetTitleSize( 0.15)
    hRatioUp.GetXaxis().SetLabelSize(0.085)
    hRatioUp.GetYaxis().CenterTitle()
    hRatioUp.Divide( unfoldHisto )
    hRatioUp.GetYaxis().SetRangeUser(0.3,1.7)
    hRatioUp.Draw('P0')
    
    #tmpPad2.SetLabelSize(16, 'x')
    #tmpPad2.SetTitleSize(19, 'x')
    #tmpPad2.SetLabelSize(16., 'y')
    #tmpPad2.SetTitleSize(19., 'y')
    #tmpPad2.SetNdivisions(505, 'x')
    #tmpPad2.SetNdivisions(505, 'y')
    #tmpPad2.SetMarkerStyle(8)
    #hRatioUp.SetTitle("#tau_{1}^{(1)}")
    #hRatioUp.SetLabelSize(18)
    
    #hRatioUp.GetXaxis().SetTitle( genJetHisto.GetXaxis().GetTitle() )
    pad2.Modified()

    #pad2.Update()
    #pad2.Modified()
    #hRatioDown.Draw('P same')
    #pad1.cd()
    #leg.Draw()
    #can.Update()
    #can.Modified()
    can.Draw()
    can.SaveAs('Plots/'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_'+version+'.png')

    canUnc = ROOT.TCanvas('canUnc', 'canUnc',  10, 10, 750, 500 )
    canUnc.SetTitle("")
    #canUnc.SetLogy()
    #ROOT.gStyle.SetOptStat(0)
    legend=ROOT.TLegend(0.78,0.65,0.92,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)
    legend.SetLineStyle(0)
    legend.AddEntry( unfoldHistoTotUnc, 'Total Unc', 'l' )

    unfoldHistoTotUnc.SetStats(0)
    unfoldHistoTotUnc.SetLineWidth(2)
    unfoldHistoTotUnc.Scale( 1/(unfoldHistoTotUnc.Integral()) )
    unfoldHistoTotUnc.SetMaximum(1.)
    unfoldHistoTotUnc.SetMinimum(0.000001)
    unfoldHistoTotUnc.GetYaxis().SetTitle('Fractional Uncertainty')
    unfoldHistoTotUnc.Draw('hist')

    if len(sysUncert)>0 :
        legend.AddEntry( unfoldHistoSysUnc, 'Total Syst Unc', 'l' )
        unfoldHistoSysUnc.Scale( 1/unfoldHistoTotUnc.Integral() )
        unfoldHistoSysUnc.SetLineWidth(2)
        unfoldHistoSysUnc.SetStats(0)
        unfoldHistoSysUnc.Draw("hist same")
        dummy=2
        for k in uncerUnfoldHisto:
            if k.endswith('Total'):
                for i in sysUncert:
                    if i in k:
                        legend.AddEntry( uncerUnfoldHisto[k], i.split('_')[1]+" Total", 'l' )
                uncerUnfoldHisto[k].SetLineColor(dummy)
                uncerUnfoldHisto[k].SetLineWidth(2)
                uncerUnfoldHisto[k].SetStats(0)
                uncerUnfoldHisto[k].Scale( 1/unfoldHistoTotUnc.Integral() )
                uncerUnfoldHisto[k].Draw("hist same")
                dummy=dummy+1

    legend.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    ROOT.gPad.SetLogy()
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(canUnc, 4, 0)
    canUnc.Draw()
    canUnc.SaveAs('Plots/'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_UNC_'+version+'.png')


|------> Unfolding _tau_1_0
|------> Cross check: plotting data vs all MC
('Processing.......', '_tau_1_0_fromData_TT_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png')
('Processing.......', '_tau_1_0_fromData_TT_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png')
|------> Cross check: plotting response matrix for signal
35.92 fb^{-1}, 2016 (13 TeV)
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding starts:
|------> TUnfolding adding input:
|------> TUnfolding adding uncert:
_jesTotalUp
35.92 fb^{-1}, 2016 (13 TeV)
_jesTotalDown
35.92 fb^{-1}, 2016 (13 TeV)
_jerUp
35.92 fb^{-1}, 2016 (13 TeV)
_jerDown
35.92 fb^{-1}, 2016 (13 TeV)
_puUp
35.92 fb^{-1}, 2016 (13 TeV)
_puDown
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
35.92 fb^{-1},  2016 (13 TeV)
35.92 fb^{-1},  2016 (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_1_0_fromData_TT_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_1_0_fromData_TT_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_1_0_fromData_TT_topSel_responseMatrixv01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_1_0_fromData_TT_topSelNormalized_responseMatrixv01_fin.png has been created
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 3 output bins do not depend on the input data  0 11-12
Info in <TUnfold::TUnfold>: 22 input bins and 10 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:[ufl])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #11 (xaxis:[0.62,0.8])


In [41]:
h=0
for i in signalHistos.keys():
    #print i
    if 'resp' in i and 'nom' in i and not 'Norm' in i:
        h = signalHistos[i]
        
pur, stab = purity_stability(h0=h, bins=np.array(variables[variables.keys()[0]]))

can_ps = ROOT.TCanvas('can_ps', 'can_ps',  10, 10, 750, 500 )
can_ps.SetTitle("")
#can_ps.SetLogy()
#ROOT.gStyle.SetOptStat(0)
legend=ROOT.TLegend(0.25,0.25,0.45,0.48)
legend.SetFillStyle(0)
legend.SetTextSize(0.03)
legend.SetBorderSize(0)
legend.SetLineStyle(0)
legend.AddEntry( pur, 'Purity', 'l' )

pur.SetStats(0)
pur.SetLineWidth(2)
pur.SetLineColor(ROOT.kRed-2)
pur.SetLineStyle(1)
pur.GetYaxis().SetTitle('Purity/Stability')
pur.GetXaxis().SetTitle('#tau_{1}^{(1)}')
pur.Draw('hist')

stab.SetStats(0)
stab.SetLineWidth(2)
stab.SetLineColor(ROOT.kBlue+2)
stab.SetLineStyle(2)
stab.GetYaxis().SetTitle('Purity/Stability')
stab.Draw('hist same')
legend.AddEntry( stab, 'Stability', 'l' )

legend.Draw()
CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
#ROOT.gPad.SetLogy()
CMS_lumi.relPosX = 0.11
CMS_lumi.CMS_lumi(can_ps, 4, 0)
can_ps.Draw()
can_ps.SaveAs('Plots/'+variables.keys()[0]+sel+'_from'+'_PurityandStability_'+version+'.png')


35.92 fb^{-1},  2016 (13 TeV)


/shome/kdatta/.local/lib/python2.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in divide
/shome/kdatta/.local/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in divide
Warning in <TCanvas::Constructor>: Deleting canvas with same name: can_ps
Info in <TCanvas::Print>: png file Plots/_tau_1_0_topSel_from_PurityandStability_v01_fin.png has been created


In [42]:
variables = {}
#variables[ '_tau_0p5_0' ] = [10] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_0' ] = array('d',[0., 0.1, 0.16, 0.22, 0.28, 0.34, 0.4, 0.46, 0.52, 0.58, 0.64, 0.8])#[0. 0.25 0.30625 0.3625 0.418750.475 0.53125 0.5875 0.64375 0.7 0.8 ][0., 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.65]#[ 6]
variables[ '_tau_2_0' ] = array('d',[0., 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5])
#variables[ '_tau_0p5_1' ] = [10] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_1' ] = [ 8]
#variables[ '_tau_2_1' ] = [ 5]
#variables[ '_tau_0p5_2' ] = [8] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_2' ] = [ 5]
#variables[ '_tau_2_2' ] = [ 4]

#variables[ '_tau_0p5_3' ] = [8] #gen_bins2  ### (reco,gen)
#variables[ '_tau_1_3' ] = [ 5]
#variables[ '_tau_2_3' ] = [0., 0.006, 0.012, 0.018, 0.024, 0.03, 0.04, 0.055, 0.075, 0.11] #[4]#[0., 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02, 0.0225, 0.025]

sysUncert = [ '_jesTotal', '_jer', '_pu' ]
print variables

{'_tau_2_0': array('d', [0.0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5])}


In [43]:
sel= '_topSel'
process = 'data'
version = 'v01_fin'

In [44]:
dataFile = {}
dataFile['data'] = [ TFile('Histos_SysUnc/jetObservables_histograms_SingleMuonRun2016ALL.root' ), 'Data', 'kBlack' ]

In [45]:
sigFiles = {}
sigFiles['TT'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TT_TuneCUETP8M2T4_13TeV-powheg-pythia8.root'), 'ttbar (powheg)', 'kRed' ]
#sigFiles['TTJets'] = [ TFile('Histos_SysUnc/jetObservables_histograms_TTJets_TuneCUETP8M1_13TeV-madgraphMLM-pythia8.root'), 'ttbar (MG)', 'kBlue' ]

In [46]:
bkgFiles = {}
bkgFiles['ST_s-channel_4f_InclusiveDecays'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_s-channel_4f_InclusiveDecays_13TeV-amcatnlo-pythia8.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_antitop'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_t-channel_antitop_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_t-channel_top'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_t-channel_top_4f_inclusiveDecays_13TeV-powhegV2-madspin-pythia8_TuneCUETP8M1.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_antitop'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M2T4.root' ), 'Single top', 'kMagenta' ]
bkgFiles['ST_tW_top'] = [ TFile('Histos_SysUnc/jetObservables_histograms_ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M2T4.root' ), 'Single top', 'kMagenta' ]
#bkgFiles['WJets'] = [ TFile( inputFolder+'/jetObservables_histograms_WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8.root' ), 'WJets', 'kCyan' ]
###bkgFiles[] = [ '', TFile( inputFolder+'/jetObservables_histograms_'+ibkg+'.root' ), '', 'kMagenta' ]

In [47]:
dataHistos = loadHistograms( dataFile, variables, sel='_topSel', isMC=False )
signalHistos = loadHistograms( sigFiles, variables, sel='_topSel', sysUnc=sysUncert )
bkgHistos = loadHistograms( bkgFiles, variables, sel='_topSel', sysUnc=sysUncert ) 

('New variable bins:', array('d', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5]), 'recoJet_tau_2_0_nom_topSel', array('d', [0.0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5]))
('New variable bins:', array('d', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5]), 'recoJet_tau_2_0_nom_topSel', array('d', [0.0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5]))
('New variable bins:', array('d', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5]), 'recoJet_tau_2_0_jesTotalDown_topSel', array('d', [0.0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5]))
('New variable bins:', array('d', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5]), 'recoJet_tau_2_0_jerDown_topSel', array('d', [0.0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5]))
('New variable bins:', array('d', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5]), 'recoJet_ta

('New variable bins:', array('d', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5]), 'recoJet_tau_2_0_puUp_topSel', array('d', [0.0, 0.08, 0.14, 0.2, 0.26, 0.32, 0.38, 0.5]))
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5])
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5])
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5])
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5])
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5])
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.44, 0.5])
('New variable bins TH2F:', [0.0, 0.04, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.2

In [48]:
for ivar in variables:
    
    print '|------> Unfolding '+ivar

    ######## Cross check: plotting data vs all MC Scaled
    print '|------> Cross check: plotting data vs all MC'
    allBkgHisto = dataHistos['data_recoJet'+ivar+'_nom'+sel].Clone()
    allBkgHisto.Reset()
    allBkgHistoNorm = allBkgHisto.Clone()
    allBkgHistoGenBin = dataHistos['data_recoJet'+ivar+'_nom'+sel+'_genBin'].Clone()
    allBkgHistoGenBin.Reset()
    for ibkg in bkgHistos:
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel): allBkgHisto.Add( bkgHistos[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_genBin'): allBkgHistoGenBin.Add( bkgHistos[ibkg].Clone() )
        if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'): allBkgHistoNorm.Add( bkgHistos[ibkg].Clone() )
    allMCHisto = allBkgHisto.Clone()
    allMCHisto.Add( signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel ].Clone() )
    plotSimpleComparison( dataHistos['data_recoJet'+ivar+'_nom'+sel].Clone(), 'data', allMCHisto, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom", rebinX=variables[ivar][0], version=sel+'_'+version  )
    allMCHistoNorm = allBkgHistoNorm.Clone()
    allMCHistoNorm.Add( signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized" ].Clone() )
    plotSimpleComparison( dataHistos['data_recoJet'+ivar+'_nom'+sel+"_Normalized"].Clone(), 'data', allMCHistoNorm, 'allBkgs', ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+"_nom_Normalized", rebinX=variables[ivar][0], version=sel+'_'+version  )

    ######## Cross check: plotting response matrix
    #tdrStyle.SetPadRightMargin(0.12)
    print '|------> Cross check: plotting response matrix for signal'
    can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750,600 )
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].SetStats(0)
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel].Draw("colz")
    can2D.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can2D, 4, 0)
    can2D.Draw()
    can2D.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'_responseMatrix'+version+'.png')
    can2DNorm = ROOT.TCanvas(ivar+'can2DNorm', ivar+'can2DNorm', 750, 600 )
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].SetStats(0)
    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'].Draw("colz")
    can2DNorm.SetRightMargin(0.13)
    CMS_lumi.extraText = "  Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
    #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(can2DNorm, 4, 0)
    can2DNorm.Draw()
    can2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+'Normalized_responseMatrix'+version+'.png')

    ####### Removing bkgs from data
    dataMinusBkgs = dataHistos[ 'data_recoJet'+ivar+'_nom'+sel ].Clone()
    dataMinusBkgs.Add( allBkgHisto.Clone(), -1 )
    dataMinusBkgs.Scale( 1/dataMinusBkgs.Integral() )
    dataMinusBkgsGenBin = dataHistos[ 'data_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
    dataMinusBkgsGenBin.Add( allBkgHistoGenBin.Clone(), -1 )
    dataMinusBkgsGenBin.Scale( 1/dataMinusBkgsGenBin.Integral() )

    ######## TUnfold part
    print '|------> TUnfolding starts:'

    ##### Defining options for TUnfold
    tunfolder = ROOT.TUnfoldDensity(
                                        signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel+'_Normalized'], ### response matrix
                                        #signalHistos[next(iter(sigFiles))+'_respJet'+ivar+'_nom'+sel], ### response matrix
                                        ROOT.TUnfold.kHistMapOutputHoriz,  #### kHistMapOutputVert if x->reco and y->gen, kHistMapOutputHoriz if x->gen and y->reco
                                        ROOT.TUnfold.kRegModeCurvature,   ##### Regularization Mode : ROOT.TUnfold.kRegModeCurvature regularizes based on the 2nd derivative of the output. More information wrt the other options can be gained from reading the source code
                                        ROOT.TUnfold.kEConstraintNone,    ##### Constraint : TUnfold.kEConstraintNone meaning we do not constrain further, the other option is to force constraint of area. (Need to look into this!!)
                                        ROOT.TUnfoldDensity.kDensityModeBinWidth  ##### Density Mode: ROOT.TUnfoldDensity.kDensityModeBinWidth uses the bin width to normalize the event rate in a given bin, accounting for non-uniformity in bin widths as discussed in section 7.2.1 of the TUnfold paper
                                        )

    ##### Defining input (data recoJet )
    print '|------> TUnfolding adding input:'
    #tunfolder.SetInput( dataHistos[ 'data_recoJet'+ivar+'_nom'+sel ].Clone() )
    tunfolder.SetInput( dataMinusBkgs )

    ###### Removing bkgs from data using TUnfold. Better to subtract bkgs beforehand
    #for ibkg in bkgHistos:
    #    if ibkg.endswith('_recoJet'+ivar+'_nom'+sel+'_Normalized'):
    #        print '|--------> Removing this bkg: ', ibkg
    #        tunfolder.SubtractBackground( bkgHistos[ibkg], ibkg )

    ###### Adding SYS unc
    if len(sysUncert)>0 :
        print '|------> TUnfolding adding uncert:'
        for sys in sysUncert:
            #plotSysComparison( signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+"_Normalized"],
            #                    signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Up'+sel+"_Normalized"],
            #                    signalHistos[next(iter(sigFiles))+'_recoJet'+ivar+sys+'Down'+sel+"_Normalized"],
            #                    ivar+'_'+next(iter(sigFiles)),
            #                    sys.split('_')[1],
            #                    version=sel+'_'+version
            #                    )
            for upDown in [ 'Up', 'Down' ]:
                print sys+upDown
                tunfolder.AddSysError(
                                    signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"],
                                    sys+upDown,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfoldSys.kSysErrModeMatrix, #### kSysErrModeMatrix the histogram sysError corresponds to an alternative response matrix. kSysErrModeShift the content of the histogram sysError are the absolute shifts of the response matrix. kSysErrModeRelative the content of the histogram sysError specifies the relative uncertainties
                                    )
                can2DNorm = ROOT.TCanvas(ivar+'can2DNorm'+sys+upDown, ivar+'can2DNorm'+sys+upDown, 750, 600 )
                signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].SetStats(0)
                signalHistos[next(iter(sigFiles))+'_respJet'+ivar+sys+upDown+sel+"_Normalized"].Draw("colz")
                can2DNorm.SetRightMargin(0.13)
                CMS_lumi.extraText = "  Preliminary"
                CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1}, 2016"
                #CMS_lumi.lumi_13TeV = "13 TeV, 2016"
                CMS_lumi.relPosX = 0.11
                CMS_lumi.CMS_lumi(can2DNorm, 4, 0)
                can2DNorm.Draw()
                can2DNorm.SaveAs('Plots/'+ivar+'_from'+('Data' if process.startswith('data') else 'MC')+'_'+next(iter(sigFiles))+sel+sys+upDown+'Normalized_responseMatrix'+version+'.png')

    ###### Running the unfolding
    print '|------> TUnfolding doUnfold:'
    tunfolder.DoUnfold(0)

    ###### Regularization
#        nScan=50
#        tauMin=0.0
#        tauMax=0.0
#        iBest=0
#
#        logTauX = ROOT.MakeNullPointer(ROOT.TSpline)
#        logTauY = ROOT.MakeNullPointer(ROOT.TSpline)
#        lCurve = ROOT.MakeNullPointer(ROOT.ROOT.TGraph)
#        ## this method scans the parameter tau and finds the kink in the L curve finally, the unfolding is done for the best choice of tau
#        tunfolder.ScanLcurve(nScan,tauMin,tauMax,lCurve,logTauX,logTauY)
    #########################
    ROOT.gStyle.SetOptStat(0)

    ##### Get output of unfolding
    unfoldHisto = tunfolder.GetOutput("unfoldHisto")

    #### Get various covariances
    print '|------> TUnfolding covariances'
    cov = tunfolder.GetEmatrixTotal("cov", "Covariance Matrix")
    cov_uncorr = tunfolder.GetEmatrixSysUncorr("cov_uncorr", "Covariance Matrix from Uncorrelated Uncertainties")
    cov_uncorr_data = tunfolder.GetEmatrixInput("cov_uncorr_data", "Covariance Matrix from Stat Uncertainties of Input Data")
    unfoldHistowoUnc = unfoldHisto.Clone()        # Unfolding and stat unc
    unfoldHistoStatUnc = unfoldHisto.Clone("unfoldHistoStatUnc")        # Unfolding and stat unc
    unfoldHistoTotUnc = unfoldHisto.Clone("unfoldHistoTotUnc")          # Total uncertainty
    for ibin in range( 0, unfoldHisto.GetNbinsX()+1 ):
        unc_tot = ROOT.TMath.Sqrt( cov.GetBinContent(ibin,ibin) )
        unfoldHistoTotUnc.SetBinContent(ibin, unc_tot )
        unfoldHisto.SetBinError(ibin, unc_tot )


    ##### Get systematic shifts of output
    uncerUnfoldHisto = {}
    if len(sysUncert)>0 :
        print '|------> TUnfolding uncertainties:'
        unfoldHistoSysUnc = unfoldHisto.Clone("unfoldHistoSysUnc")          # Syst uncertainty
        unfoldHistoSysUnc.Reset()
        unfoldHistoSysUnc.SetLineStyle(2)

        for sys in sysUncert:
            for upDown in [ 'Up', 'Down' ]:
                print sys+upDown
                uncerUnfoldHisto[ivar+sys+upDown] = tunfolder.GetDeltaSysSource(sys+upDown, "unfoldHisto_"+sys+upDown+"shift", "-1#sigma")
                try: uncerUnfoldHisto[ivar+sys+upDown].SetLineStyle(2)
                except ReferenceError: uncerUnfoldHisto.pop( ivar+sys+upDown, None )

            # Create total uncertainty and sys uncertainty plots.
            uncerUnfoldHisto[ivar+sys+'Total'] = unfoldHisto.Clone("unfoldHistoSysUnc")          # Syst uncertainty
            uncerUnfoldHisto[ivar+sys+'Total'].Reset()
            uncerUnfoldHisto[ivar+sys+'Total'].SetLineStyle(3)
            for i in xrange( 0, unfoldHisto.GetNbinsX() + 1):
                try: yup = abs( uncerUnfoldHisto[ivar+sys+'Up'].GetBinContent(i))
                except KeyError: yup = 0
                try: ydn = abs( uncerUnfoldHisto[ivar+sys+'Down'].GetBinContent(i))
                except KeyError: ydn = 0
                dy = ROOT.TMath.Sqrt( (yup**2 + ydn**2) )
                uncerUnfoldHisto[ivar+sys+'Total'].SetBinContent(i, dy )
            unfoldHistoSysUnc.Add( uncerUnfoldHisto[ivar+sys+'Total'] )

    ###### Plot unfolding results
    #tdrStyle.SetPadRightMargin(0.05)
    #tdrStyle.SetPadLeftMargin(0.15)
    can = ROOT.TCanvas('can', 'can',  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()
    #ROOT.gStyle.SetOptStat(0)

    pad1.cd()
    
    #tmpPad1 = pad1.DrawFrame( 0, 0.3, 1, 1.7 )
    
    #tmpPad1.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(2)})}' )
    #tmpPad1.GetYaxis().SetTitleOffset(0.9)
    #tmpPad1.GetYaxis().SetTitleSize(28)
    #tmpPad1.GetYaxis().SetLabelSize(20)
    #tmpPad1.SetStats(0)
    
    leg=ROOT.TLegend(0.65,0.6,0.95,0.88)
    leg.SetLineColor(0)
    leg.SetLineWidth(0)
    leg.SetBorderSize(0)
    leg.SetFillStyle(0)
    #leg.SetTextSize(0.8)

    #genJetHisto = signalHistos[ next(iter(sigFiles))+'_genJet'+ivar+sel ].Clone()
    genJetHisto = signalHistos[ next(iter(sigFiles))+'_genJet'+ivar+sel+'_Normalized' ].Clone()
    genJetHisto.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(#tau_{1}^{(2)})}' )
    genJetHisto.GetYaxis().SetTitleSize( 0.05 )
    genJetHisto.GetYaxis().SetTitleOffset( 0.9 )
    #genJetHisto.GetYaxis().CenterTitle()
    genJetHisto.SetStats(0)
    genJetHisto.Scale(1, 'width')  ### divide by bin width
    genJetHisto.SetLineWidth(2)
    genJetHisto.GetYaxis().SetLabelSize(0.037)
    genJetHisto.SetLineColor(1)
    pad1.Modified()
    pad1.Update()

    leg.AddEntry( genJetHisto, 'Gen-level MC', 'l' )

    #unfoldHisto = tunfolder.GetOutput("unfolded")
    #if len(variables[ivar])>1: unfoldHisto = unfoldHisto.Rebin( len(variables[ivar])-1, unfoldHisto.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    #else: unfoldHisto.Rebin( variables[ivar][0] )
    

    unfoldHistowoUnc.Scale(1, 'width')  ### divide by bin width
    unfoldHistowoUnc.SetMarkerStyle(0)
    unfoldHistowoUnc.SetMarkerColor(ROOT.kRed-5)
    unfoldHistowoUnc.SetLineColor(ROOT.kRed-5)
    unfoldHistowoUnc.SetLineWidth(2)
    unfoldHistowoUnc.SetStats(0)
    leg.AddEntry( unfoldHistowoUnc, 'Unfolded, stat+unf unc', 'l' )
    
    unfoldHisto.Scale(1, 'width')  ### divide by bin width
    unfoldHisto.SetMarkerStyle(4)
    unfoldHisto.SetMarkerSize(2)
    unfoldHisto.SetMarkerColor(ROOT.kRed+1)
    unfoldHisto.SetLineColor(ROOT.kRed+1)
    unfoldHisto.SetStats(0)
    leg.AddEntry( unfoldHisto, 'Unfolded, total unc', 'pl' )

    foldHisto = tunfolder.GetFoldedOutput("folded") #, 'folded', 'folded', , False)
    if len(variables[ivar])>1: foldHisto = foldHisto.Rebin( len(variables[ivar])-1, foldHisto.GetName()+"_rebin", array( 'd', variables[ivar] ) )
    else: foldHisto.Rebin( 2 )
    foldHisto.Scale(1, 'width')  ### divide by bin width
    foldHisto.SetLineWidth(2)
    foldHisto.SetLineStyle(2)
    foldHisto.SetLineColor(8)
    foldHisto.SetStats(0)
    leg.AddEntry( foldHisto, 'Folded', 'l' )

    if process.startswith('data'):
        #recoJetHisto = signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_Normalized' ].Clone()
        recoJetHisto = signalHistos[ next(iter(sigFiles))+'_recoJet'+ivar+'_nom'+sel+'_genBin' ].Clone()
        recoJetHisto.Scale(1/recoJetHisto.Integral(), 'width')  ### divide by bin width
        recoJetHisto.SetLineWidth(2)
        recoJetHisto.SetLineStyle(2)
        recoJetHisto.SetLineColor(ROOT.kBlue)
        leg.AddEntry( recoJetHisto, 'Reco-level MC', 'l' )

    dataJetHisto = dataMinusBkgsGenBin.Clone()
    dataJetHisto.Scale(1/dataJetHisto.Integral(), 'width')  ### divide by bin width
    dataJetHisto.SetLineWidth(2)
    dataJetHisto.SetLineStyle(2)
    dataJetHisto.SetLineColor(ROOT.kMagenta)
    dataJetHisto.SetStats(0)
    leg.AddEntry( dataJetHisto, ('Data' if process.startswith('data') else 'MC Closure' ), 'l' )

    
    
    genJetHisto.SetMaximum( 1.3*max([ genJetHisto.GetMaximum(), unfoldHisto.GetMaximum(), unfoldHistowoUnc.GetMaximum(), dataJetHisto.GetMaximum() ] )  )
    pad1.Update()
    pad1.Modified()
    
    genJetHisto.Draw( "histe")
    unfoldHisto.Draw( "same")
    unfoldHistowoUnc.Draw( "e1 same")
    foldHisto.Draw( "hist same")
    dataJetHisto.Draw( "hist same")
    if process.startswith('data'): recoJetHisto.Draw( "hist same")
    #ROOT.gStyle.SetOptStat(0)
    leg.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    #ROOT.gPad.SetLogy()
    CMS_lumi.cmsTextSize = 1.2
    CMS_lumi.relPosX = 0.13
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    ROOT.gStyle.SetOptStat(0000000)
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    #tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1. )
    
    hRatioUp = genJetHisto.Clone()
    hRatioUp.GetYaxis().SetTitle( "#frac{MC-Truth}{Unfolded}" )
    hRatioUp.GetYaxis().SetTitleOffset( 0.6)
    hRatioUp.GetYaxis().SetTitleSize(0.1)
    hRatioUp.GetYaxis().SetLabelSize(0.085)
    hRatioUp.GetXaxis().SetTitle("#tau_{1}^{(2)}")
    hRatioUp.GetXaxis().SetTitleOffset( 0.8)
    hRatioUp.GetXaxis().SetTitleSize( 0.15)
    hRatioUp.GetXaxis().SetLabelSize(0.085)
    hRatioUp.GetYaxis().CenterTitle()
    hRatioUp.Divide( unfoldHisto )
    hRatioUp.GetYaxis().SetRangeUser(0.3,1.7)
    hRatioUp.Draw('P0')
    
    #tmpPad2.SetLabelSize(16, 'x')
    #tmpPad2.SetTitleSize(19, 'x')
    #tmpPad2.SetLabelSize(16., 'y')
    #tmpPad2.SetTitleSize(19., 'y')
    #tmpPad2.SetNdivisions(505, 'x')
    #tmpPad2.SetNdivisions(505, 'y')
    #tmpPad2.SetMarkerStyle(8)
    #hRatioUp.SetTitle("#tau_{1}^{(2)}")
    #hRatioUp.SetLabelSize(18)
    
    #hRatioUp.GetXaxis().SetTitle( genJetHisto.GetXaxis().GetTitle() )
    pad2.Modified()

    #pad2.Update()
    #pad2.Modified()
    #hRatioDown.Draw('P same')
    #pad1.cd()
    #leg.Draw()
    #can.Update()
    #can.Modified()
    can.Draw()
    can.SaveAs('Plots/'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_'+version+'.png')

    canUnc = ROOT.TCanvas('canUnc', 'canUnc',  10, 10, 750, 500 )
    canUnc.SetTitle("")
    #canUnc.SetLogy()
    #ROOT.gStyle.SetOptStat(0)
    legend=ROOT.TLegend(0.8,0.65,0.94,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.SetBorderSize(0)
    legend.SetLineStyle(0)
    legend.AddEntry( unfoldHistoTotUnc, 'Total Unc ', 'l' )

    unfoldHistoTotUnc.SetStats(0)
    unfoldHistoTotUnc.SetLineWidth(2)
    unfoldHistoTotUnc.Scale( 1/(unfoldHistoTotUnc.Integral()) )
    unfoldHistoTotUnc.SetMaximum(1.)
    unfoldHistoTotUnc.SetMinimum(0.000001)
    #unfoldHistoTotUnc.SetMaximum(unfoldHistoTotUnc.GetMaximum())
    unfoldHistoTotUnc.GetYaxis().SetTitle('Fractional Uncertainty')
    unfoldHistoTotUnc.Draw('hist')

    if len(sysUncert)>0 :
        legend.AddEntry( unfoldHistoSysUnc, 'Total Syst Unc', 'l' )
        unfoldHistoSysUnc.Scale( 1/unfoldHistoTotUnc.Integral() )
        unfoldHistoSysUnc.SetLineWidth(2)
        unfoldHistoSysUnc.SetStats(0)
        unfoldHistoSysUnc.Draw("hist same")
        dummy=2
        for k in uncerUnfoldHisto:
            if k.endswith('Total'):
                for i in sysUncert:
                    if i in k:
                        legend.AddEntry( uncerUnfoldHisto[k], i.split('_')[1]+" Total", 'l' )
                uncerUnfoldHisto[k].SetLineColor(dummy)
                uncerUnfoldHisto[k].SetLineWidth(2)
                uncerUnfoldHisto[k].SetStats(0)
                uncerUnfoldHisto[k].Scale( 1/unfoldHistoTotUnc.Integral() )
                uncerUnfoldHisto[k].Draw("hist same")
                dummy=dummy+1

    legend.Draw()
    CMS_lumi.extraText = "Preliminary"
    CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
    ROOT.gPad.SetLogy()
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(canUnc, 4, 0)
    canUnc.Draw()
    canUnc.SaveAs('Plots/'+ivar+sel+'_from'+('Data' if process.startswith('data') else 'MC')+(''.join(sysUncert))+'_Tunfold_UNC_'+version+'.png')


|------> Unfolding _tau_2_0
|------> Cross check: plotting data vs all MC
('Processing.......', '_tau_2_0_fromData_TT_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png')
('Processing.......', '_tau_2_0_fromData_TT_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png')
|------> Cross check: plotting response matrix for signal
35.92 fb^{-1}, 2016 (13 TeV)
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding starts:
|------> TUnfolding adding input:
|------> TUnfolding adding uncert:
_jesTotalUp
35.92 fb^{-1}, 2016 (13 TeV)
_jesTotalDown
35.92 fb^{-1}, 2016 (13 TeV)
_jerUp
35.92 fb^{-1}, 2016 (13 TeV)
_jerDown
35.92 fb^{-1}, 2016 (13 TeV)
_puUp
35.92 fb^{-1}, 2016 (13 TeV)
_puDown
35.92 fb^{-1}, 2016 (13 TeV)
|------> TUnfolding doUnfold:
|------> TUnfolding covariances
|------> TUnfolding uncertainties:
_jesTotalUp
_jesTotalDown
_jerUp
_jerDown
_puUp
_puDown
35.92 fb^{-1},  2016 (13 TeV)
35.92 fb^{-1},  2016 (13 TeV)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_2_0_fromData_TT_nom_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file Plots/_tau_2_0_fromData_TT_nom_Normalized_dataallBkgs_simpleComparisonPlot_topSel_v01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_2_0_fromData_TT_topSel_responseMatrixv01_fin.png has been created
Info in <TCanvas::Print>: png file Plots/_tau_2_0_fromData_TT_topSelNormalized_responseMatrixv01_fin.png has been created
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 3 output bins do not depend on the input data  0 7-8
Info in <TUnfold::TUnfold>: 14 input bins and 6 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:[ufl])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #7 (xaxis:[0.38,0.5])
Info

In [49]:
h=0
for i in signalHistos.keys():
    #print i
    if 'resp' in i and 'nom' in i and not 'Norm' in i:
        h = signalHistos[i]
        
pur, stab = purity_stability(h0=h, bins=np.array(variables[variables.keys()[0]]))

can_ps = ROOT.TCanvas('can_ps', 'can_ps',  10, 10, 750, 500 )
can_ps.SetTitle("")
#can_ps.SetLogy()
#ROOT.gStyle.SetOptStat(0)
legend=ROOT.TLegend(0.25,0.25,0.45,0.48)
legend.SetFillStyle(0)
legend.SetTextSize(0.03)
legend.SetBorderSize(0)
legend.SetLineStyle(0)
legend.AddEntry( pur, 'Purity', 'l' )

pur.SetStats(0)
pur.SetLineWidth(2)
pur.SetLineColor(ROOT.kRed-2)
pur.SetLineStyle(1)
pur.GetYaxis().SetTitle('Purity/Stability')
pur.GetXaxis().SetTitle('#tau_{1}^{(2)}')
pur.Draw('hist')

stab.SetStats(0)
stab.SetLineWidth(2)
stab.SetLineColor(ROOT.kBlue+2)
stab.SetLineStyle(2)
stab.GetYaxis().SetTitle('Purity/Stability')
stab.Draw('hist same')
legend.AddEntry( stab, 'Stability', 'l' )

legend.Draw()
CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 2 ) )+" fb^{-1},  2016"
#ROOT.gPad.SetLogy()
CMS_lumi.relPosX = 0.11
CMS_lumi.CMS_lumi(can_ps, 4, 0)
can_ps.Draw()
can_ps.SaveAs('Plots/'+variables.keys()[0]+sel+'_from'+'_PurityandStability_'+version+'.png')

35.92 fb^{-1},  2016 (13 TeV)


/shome/kdatta/.local/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in divide
Warning in <TCanvas::Constructor>: Deleting canvas with same name: can_ps
Info in <TCanvas::Print>: png file Plots/_tau_2_0_topSel_from_PurityandStability_v01_fin.png has been created
